In [1]:
import json
from collections import Counter
import pandas as pd

# Path to your file
file_path = "/nlp/data/karthik9/Sprint1/Dental/Data/jsons/dichotomous_outcomes.json"

# Load the JSON
with open(file_path, "r") as f:
    data = json.load(f)

# Count frequency of Ref_ID
ref_counts = Counter(record["Ref_ID"] for record in data)

# Convert to DataFrame for easy display
df = pd.DataFrame(ref_counts.items(), columns=["Ref_ID", "Count"])
df = df.sort_values(by="Count", ascending=True).head(20).reset_index(drop=True)

df

,Ref_ID,Count
0,1789,2
1,2191,2
2,1759,3
3,1003,4
4,574,4
5,4253,4
6,3869,4
7,3152,4
8,348,4
9,10,4


In [1]:
# class MedicalExtractionEvaluator:
#     """Enhanced evaluator for medical data extraction quality with precision, recall, accuracy metrics, and semantic matching."""
    
#     def __init__(self, use_semantic=True, semantic_threshold=0.8):
#         self.required_fields = [
#             'First_Author', 'Population',
#             'Intervention_Code', 'Intervention_Description', 'Outcome_Type',
#             'Follow_Up_Time', 'N_Analyzed', 'Adverse_Effect_Specify',
#             'N_Events_Number', 'N_Events_Percentage', 'Comments'
#         ]
        
#         # Semantic matching setup
#         self.use_semantic = use_semantic
#         self.semantic_threshold = semantic_threshold
        
#         if self.use_semantic:
#             # Use a lightweight model for cost efficiency
#             self.semantic_model = SentenceTransformer('all-MiniLM-L6-v2')  # Fast, small model
            
#             # Fields that benefit from semantic matching
#             self.semantic_fields = [
#                 'Intervention_Description', 
#                 'Adverse_Effect_Specify',
#                 'Follow_Up_Time'
#             ]
            
#             # Fields that need exact matching
#             self.exact_fields = [
#                 'First_Author',
#                 'Population',
#                 'Intervention_Code',
#                 'Outcome_Type',
#                 'N_Analyzed',
#                 'N_Events_Number',
#                 'N_Events_Percentage',
#             ]
    
#     def normalize_value(self, value: Any) -> str:
#         """Normalize values for comparison."""
#         if value is None:
#             return ""
#         return str(value).strip().lower()
    
#     def semantic_similarity(self, text1: str, text2: str) -> float:
#         """Calculate semantic similarity between two texts."""
#         if not self.use_semantic:
#             return 1.0 if text1.lower() == text2.lower() else 0.0
            
#         if not text1.strip() or not text2.strip():
#             return 1.0 if text1.strip() == text2.strip() else 0.0
        
#         embeddings = self.semantic_model.encode([text1, text2])
#         similarity = cosine_similarity([embeddings[0]], [embeddings[1]])[0][0]
#         return float(similarity)
    
#     def field_match_score(self, extracted_value: Any, ground_truth_value: Any, field_name: str) -> float:
#         """Calculate field-level match score (exact or semantic)."""
        
#         # Handle None/empty values
#         ext_val = str(extracted_value).strip() if extracted_value is not None else ""
#         gt_val = str(ground_truth_value).strip() if ground_truth_value is not None else ""
        
#         if not self.use_semantic:
#             # Fall back to normalized exact matching
#             return 1.0 if ext_val.lower() == gt_val.lower() else 0.0
        
#         # Exact matching for certain fields
#         if field_name in self.exact_fields:
#             return 1.0 if ext_val.lower() == gt_val.lower() else 0.0
        
#         # Semantic matching for text fields
#         if field_name in self.semantic_fields:
#             if not ext_val and not gt_val:
#                 return 1.0  # Both empty
#             if not ext_val or not gt_val:
#                 return 0.0  # One empty, one not
            
#             similarity = self.semantic_similarity(ext_val, gt_val)
#             return 1.0 if similarity >= self.semantic_threshold else 0.0
        
#         # Default exact matching
#         return 1.0 if ext_val.lower() == gt_val.lower() else 0.0
    
#     def find_best_match(self, extracted_record: Dict, ground_truth_records: List[Dict]) -> Tuple[Dict, float]:
#         """Find the best matching ground truth record using semantic or exact matching."""
#         best_match = None
#         best_score = 0.0
        
#         for gt_record in ground_truth_records:
#             matches = 0
#             total_fields = 0
            
#             for field in self.required_fields:
#                 if field in extracted_record and field in gt_record:
#                     if self.use_semantic:
#                         # Use semantic matching
#                         match_score = self.field_match_score(
#                             extracted_record[field], 
#                             gt_record[field], 
#                             field
#                         )
#                         matches += match_score
#                     else:
#                         # Use original exact matching
#                         ext_val = self.normalize_value(extracted_record[field])
#                         gt_val = self.normalize_value(gt_record[field])
#                         if ext_val == gt_val:
#                             matches += 1
#                     total_fields += 1
            
#             if total_fields > 0:
#                 score = matches / total_fields
#                 if score > best_score:
#                     best_score = score
#                     best_match = gt_record
        
#         return best_match, best_score
    
#     def evaluate_completeness(self, extracted_records: List[Dict]) -> float:
#         """Evaluate field completeness."""
#         if not extracted_records:
#             return 0.0
        
#         total_fields = len(self.required_fields) * len(extracted_records)
#         filled_fields = 0
        
#         for record in extracted_records:
#             for field in self.required_fields:
#                 if field in record and record[field] is not None and str(record[field]).strip() != "":
#                     filled_fields += 1
        
#         return filled_fields / total_fields if total_fields > 0 else 0.0
    
#     def evaluate_record_metrics(self, extracted_records: List[Dict], ground_truth: List[Dict]) -> Dict[str, float]:
#         """Calculate record-level precision, recall, and F1."""
#         matched_pairs = []
#         used_gt_indices = set()
        
#         # Find matches
#         for ext_record in extracted_records:
#             best_match, best_score = self.find_best_match(ext_record, ground_truth)
#             if best_score >= 0.5:
#                 for i, gt_record in enumerate(ground_truth):
#                     if gt_record == best_match and i not in used_gt_indices:
#                         matched_pairs.append((ext_record, gt_record))
#                         used_gt_indices.add(i)
#                         break
        
#         # Calculate metrics
#         true_positives = len(matched_pairs)
#         false_positives = len(extracted_records) - true_positives
#         false_negatives = len(ground_truth) - true_positives
        
#         precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0.0
#         recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0.0
#         f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
        
#         return {
#             'precision': precision,
#             'recall': recall,
#             'f1': f1,
#             'TP': true_positives,
#             'FP': false_positives,
#             'FN': false_negatives
#         }
    
#     def evaluate_accuracy(self, extracted_records: List[Dict], ground_truth: List[Dict]) -> Dict[str, float]:
#         """Evaluate extraction accuracy against ground truth with precision, recall, and F1."""
#         if not extracted_records or not ground_truth:
#             return {"accuracy": 0.0, "precision": 0.0, "recall": 0.0, "f1": 0.0}
        
#         # Record-level metrics
#         record_metrics = self.evaluate_record_metrics(extracted_records, ground_truth)
        
#         return {
#             "precision": record_metrics['precision'],
#             "recall": record_metrics['recall'],
#             "f1": record_metrics['f1'],
#             "TP": record_metrics['TP'],
#             "FP": record_metrics['FP'],
#             "FN": record_metrics['FN'],
#             "completeness": self.evaluate_completeness(extracted_records)
#         }
    
#     def evaluate(self, extracted_records: List[Dict], ground_truth: List[Dict] = None) -> Dict[str, Any]:
#         """Complete evaluation with precision, recall, and accuracy metrics."""
#         results = {
#             "num_extracted": len(extracted_records),
#             "completeness": self.evaluate_completeness(extracted_records),
#             "semantic_enabled": self.use_semantic,
#             "semantic_threshold": self.semantic_threshold if self.use_semantic else None
#         }
        
#         if ground_truth:
#             accuracy_results = self.evaluate_accuracy(extracted_records, ground_truth)
#             results.update(accuracy_results)
#             results["num_ground_truth"] = len(ground_truth)
        
#         return results

#     def create_field_matching_matrix(self, extracted_records: List[Dict], ground_truth: List[Dict]) -> Dict:
#         """Create a field-by-field matching matrix with ground truth records as rows."""
        
#         matrix = []
#         used_extracted_indices = set()
        
#         for i, gt_record in enumerate(ground_truth):
#             row = {
#                 'gt_index': i,
#                 'gt_record_id': gt_record.get('First_Author', f'GT_{i}'),  # Use author or fallback ID
#                 'match_status': 'Missing',  # Default to missing
#                 'fields': {}
#             }
            
#             # Find best matching extracted record for this ground truth
#             best_match = None
#             best_score = 0.0
#             best_match_index = -1
            
#             for j, ext_record in enumerate(extracted_records):
#                 if j in used_extracted_indices:
#                     continue
                    
#                 # Calculate similarity score
#                 matches = 0
#                 total_fields = 0
                
#                 for field in self.required_fields:
#                     if field in gt_record and field in ext_record:
#                         match_score = self.field_match_score(
#                             ext_record[field], 
#                             gt_record[field], 
#                             field
#                         )
#                         matches += match_score
#                         total_fields += 1
                
#                 if total_fields > 0:
#                     score = matches / total_fields
#                     if score > best_score:
#                         best_score = score
#                         best_match = ext_record
#                         best_match_index = j
            
#             # If we found a match, mark it as used and compare fields
#             if best_match is not None and best_score > 0:
#                 used_extracted_indices.add(best_match_index)
#                 row['match_status'] = 'Matched'
                
#                 # Compare each field
#                 for field in self.required_fields:
#                     if field in gt_record:
#                         if field in best_match:
#                             match_score = self.field_match_score(
#                                 best_match[field], 
#                                 gt_record[field], 
#                                 field
#                             )
#                             row['fields'][field] = {
#                                 'score': match_score,
#                                 'gt_value': gt_record[field],
#                                 'extracted_value': best_match[field]
#                             }
#                         else:
#                             # Field missing in extracted
#                             row['fields'][field] = {
#                                 'score': 0.0,
#                                 'gt_value': gt_record[field],
#                                 'extracted_value': None
#                             }
#                     else:
#                         # Field not in ground truth
#                         row['fields'][field] = {
#                             'score': None,
#                             'gt_value': None,
#                             'extracted_value': best_match.get(field) if best_match else None
#                         }
#             else:
#                 # No match found - all fields are missing
#                 for field in self.required_fields:
#                     row['fields'][field] = {
#                         'score': None,  # Use None to indicate completely missing record
#                         'gt_value': gt_record.get(field),
#                         'extracted_value': None
#                     }
            
#             matrix.append(row)
        
#         return {
#             'matrix': matrix,
#             'field_names': self.required_fields,
#             'summary': {
#                 'total_gt_records': len(ground_truth),
#                 'matched_records': sum(1 for row in matrix if row['match_status'] == 'Matched'),
#                 'missing_records': sum(1 for row in matrix if row['match_status'] == 'Missing')
#             }
#         }




# # Example usage:
# # For semantic matching (default)
# evaluator = MedicalExtractionEvaluator(use_semantic=True, semantic_threshold=0.8)

In [1]:
import os
import json
import base64
import glob
import shutil

main_dir = "/nlp/data/karthik9/Sprint1/Dental/acute_pain_mds"

for subdir in os.listdir(main_dir):
    subdir_path = os.path.join(main_dir, subdir)
    if not os.path.isdir(subdir_path):
        continue

    # Find the JSON inside this subdir
    json_files = glob.glob(os.path.join(subdir_path, "*.json"))
    if not json_files:
        continue
    json_path = json_files[0]

    # Load JSON
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    images_dict = data.get("marker", {}).get("images", {})
    if not images_dict:
        print(f"No images in {json_path}")
        continue

    # Images folder
    images_dir = os.path.join(subdir_path, "images")
    os.makedirs(images_dir, exist_ok=True)

    # Delete old images
    for file in os.listdir(images_dir):
        file_path = os.path.join(images_dir, file)
        if os.path.isfile(file_path):
            os.remove(file_path)

    # Save new images
    for fname, b64_data in images_dict.items():
        try:
            img_path = os.path.join(images_dir, fname)
            img_bytes = base64.b64decode(b64_data)
            with open(img_path, "wb") as img_file:
                img_file.write(img_bytes)
            print(f"Saved {img_path}")
        except Exception as e:
            print(f"Error saving {fname} in {json_path}: {e}")


Saved /nlp/data/karthik9/Sprint1/Dental/acute_pain_mds/1003_Rayati_md/images/_page_3_Figure_15.jpeg
Saved /nlp/data/karthik9/Sprint1/Dental/acute_pain_mds/10_Zelenakas_md/images/_page_3_Figure_8.jpeg
Saved /nlp/data/karthik9/Sprint1/Dental/acute_pain_mds/10_Zelenakas_md/images/_page_4_Figure_1.jpeg
Saved /nlp/data/karthik9/Sprint1/Dental/acute_pain_mds/1102_Qi_md/images/_page_4_Figure_1.jpeg
Saved /nlp/data/karthik9/Sprint1/Dental/acute_pain_mds/1102_Qi_md/images/_page_6_Figure_1.jpeg
Saved /nlp/data/karthik9/Sprint1/Dental/acute_pain_mds/1102_Qi_md/images/_page_7_Figure_6.jpeg
Saved /nlp/data/karthik9/Sprint1/Dental/acute_pain_mds/1102_Qi_md/images/0__page_8_Figure_1.jpeg
Saved /nlp/data/karthik9/Sprint1/Dental/acute_pain_mds/1102_Qi_md/images/0__page_9_Figure_1.jpeg
Saved /nlp/data/karthik9/Sprint1/Dental/acute_pain_mds/1102_Qi_md/images/0__page_12_Figure_1.jpeg
Saved /nlp/data/karthik9/Sprint1/Dental/acute_pain_mds/1102_Qi_md/images/0__page_12_Figure_3.jpeg
Saved /nlp/data/karthik9/

In [38]:
import pandas as pd


# excel_path = "/nlp/data/karthik9/Sprint1/Dental/csvs/Acute Dental Pain Data Extraction (NMA and Pediatrics) MASTER_ASB 1 (1).xlsx"
# df = pd.read_excel(excel_path, sheet_name=2)

# Load CSV into DataFrame
df = pd.read_csv("/nlp/data/karthik9/Sprint1/Dental/csvs/updated_references.csv")
duplicates = df[df.duplicated(subset=["Ref ID"], keep=False)].sort_values("Ref ID")

print("Duplicate Filenames:")
print(duplicates["Ref ID"].unique())

Duplicate Filenames:
[]


In [39]:
import pandas as pd
import os

# Paths
csv_path = "/nlp/data/karthik9/Sprint1/Dental/csvs/updated_references.csv"
pdf_dir = "/nlp/data/karthik9/Sprint1/Dental/acute_pain_pdfs"

# Load CSV
df = pd.read_csv(csv_path)

# Get set of actual files in the directory
pdf_files = set(os.listdir(pdf_dir))

# Check presence (append .pdf if missing)
def check_pdf(fname):
    if not isinstance(fname, str) or fname.strip() == "":
        return ""
    pdf_name = fname if fname.lower().endswith(".pdf") else f"{fname}.pdf"
    if pdf_name in pdf_files:
        return os.path.join(pdf_dir, pdf_name)
    return ""

# Add new column
df["pdf_present"] = df["filename"].apply(check_pdf)

# --- Stats ---
present_count = (df["pdf_present"] != "").sum()
missing_count = (df["pdf_present"] == "").sum()

# CSV filenames not found in folder
missing_from_folder = sorted(df.loc[df["pdf_present"] == "", "filename"].dropna().unique())

# PDFs in folder not referenced in CSV
csv_filenames = set(
    df["filename"].dropna().apply(lambda x: x if x.lower().endswith(".pdf") else f"{x}.pdf")
)
extra_pdfs = sorted(pdf_files - csv_filenames)
# Print summary
print(f"✅ Total rows in CSV: {len(df)}")
print(f"📂 PDFs in folder: {len(pdf_files)}")
print(f"✔️ Present matches: {present_count}")
print(f"❌ Missing matches: {missing_count}\n")

print("CSV filenames missing in folder (with year):")
missing_rows = df.loc[df["pdf_present"] == "", ["filename", "year","doi_link","title"]].dropna(subset=["filename"])
for _, row in missing_rows.iterrows():
    print(f"  - {row['filename']} ({row['year']})")

print("\nPDFs in folder not in CSV:")
for f in extra_pdfs:
    print("  -", f)

df.to_csv(csv_path, index=False)


✅ Total rows in CSV: 84
📂 PDFs in folder: 81
✔️ Present matches: 81
❌ Missing matches: 3

CSV filenames missing in folder (with year):
  - 3840_Cooper (1988)
  - 3835_Cooper (1991)

PDFs in folder not in CSV:


In [3]:
import pandas as pd

# Read Excel sheet 3
excel_path = "/nlp/data/karthik9/Sprint1/Dental/csvs/Acute Dental Pain Data Extraction (NMA and Pediatrics) MASTER_ASB 1 (1).xlsx"
df1 = pd.read_excel(excel_path, sheet_name=2)

# Rename for consistency
df1.rename(columns={"1st Author": "first_author_lastname", "Year": "year"}, inplace=True)

# Fix year format → remove .0
df1["year"] = df1["year"].apply(lambda x: str(int(x)) if pd.notna(x) else "")
df["year"] = df["year"].astype(str).str.strip()

# Fix Ref ID format → remove .0 too
df1["Ref ID"] = df1["Ref ID"].apply(lambda x: str(int(x)) if pd.notna(x) else "")


# Normalize author names
def normalize_name(name):
    if pd.isna(name):
        return ""
    return str(name).strip().lower().replace("–", "-").replace("—", "-")

df["first_author_lastname_norm"] = df["first_author_lastname"].apply(normalize_name)
df1["first_author_lastname_norm"] = df1["first_author_lastname"].apply(normalize_name)

In [ ]:



# Merge and keep Ref ID column from df1
merged_df = df.merge(
    df1[["first_author_lastname_norm", "year", "Ref ID"]],
    left_on=["first_author_lastname_norm", "year"],
    right_on=["first_author_lastname_norm", "year"],
    how="left",
    indicator=True
)

# Create filename using cleaned Ref ID + Author
merged_df["filename"] = merged_df.apply(
    lambda x: f"{x['Ref ID']}_{x['first_author_lastname']}" if x["_merge"] == "both" else None,
    axis=1
)

# Cleanup helper columns
merged_df.drop(columns=["first_author_lastname_norm", "_merge"], inplace=True)
merged_df.to_csv("updated_references.csv", index=False, encoding="utf-8")

# Check result
print(merged_df[["first_author_lastname", "year", "filename"]].head(10))


  first_author_lastname  year        filename
0              Ahlström  1993            None
1             Al-Sukhun  2012  4526_Al-Sukhun
2                Bakshi  1994     4253_Bakshi
3               Bentley  1987    4153_Bentley
4                 Black  2002      4105_Black
5                 Chang  2005      3873_Chang
6                 Chang  2004      3870_Chang
7                 Chang  2004      3872_Chang
8                 Chang  2004      3870_Chang
9                 Chang  2004      3872_Chang


In [33]:
# Count unique and duplicate filenames
total_filenames = merged_df["filename"].notna().sum()
unique_filenames = merged_df["filename"].nunique()
duplicate_count = total_filenames - unique_filenames

print(f"Total filenames: {total_filenames}")
print(f"Unique filenames: {unique_filenames}")
print(f"Duplicate filenames: {duplicate_count}")

# List duplicates
duplicates = merged_df[merged_df.duplicated("filename", keep=False)]
duplicate_list = duplicates["filename"].unique()

print("\nDuplicate Filenames:")
print(duplicate_list)

# List unique filenames only
unique_list = merged_df["filename"].dropna().drop_duplicates()

print("\nUnique Filenames:")
print(unique_list.tolist())


Total filenames: 87
Unique filenames: 73
Duplicate filenames: 14

Duplicate Filenames:
[None '3870_Chang' '3872_Chang' '3848_Cooper' '3840_Cooper' '3280_Forbes'
 '3245_Forbes' '2868_Hersh' '2859_Hersh' '1742_Mehlisch' '1741_Mehlisch'
 '2197_Moore' '608_Searle' '572_Seymour' '37_Yue']

Unique Filenames:
['4526_Al-Sukhun', '4253_Bakshi', '4153_Bentley', '4105_Black', '3873_Chang', '3870_Chang', '3872_Chang', '3869_Chang', '4381_Cheung', '4349_Christensen', '3848_Cooper', '3840_Cooper', '3846_Cooper', '3844_Cooper', '3841_Cooper', '3850_Cooper', '3835_Cooper', '3827_Cooper', '3287_Cooper', '3641_Daniels', '3640_Daniels', '3639_Daniels', '3593_Desjardins', '3556_Dionne', '3477_Dolci', '3280_Forbes', '3245_Forbes', '3277_Forbes', '3244_Forbes', '3213_Fricke', '3155_Gay', '3152_Gazal', '3103_Giglio', '2868_Hersh', '2859_Hersh', '2863_Hersh', '2621_Jain', '2518_Kellstein', '2412_Kiersch', '2468_Malmstrom', '2467_Malmstrom', '2466_Malmstrom', '2465_Malmstrom', '2464_Malmstrom', '1789_Matthews'

In [34]:
# Count unique & duplicate pairs of (1st Author, Year) in df1
pair_counts = df1.groupby(["first_author_lastname", "year"]).size()

# Unique pairs → appear only once
unique_pairs = pair_counts[pair_counts == 1]
# Duplicate pairs → appear more than once
duplicate_pairs = pair_counts[pair_counts > 1]

print(f"Total pairs: {len(pair_counts)}")
print(f"Unique pairs: {len(unique_pairs)}")
print(f"Duplicate pairs: {len(duplicate_pairs)}")

# List duplicate pairs explicitly
if not duplicate_pairs.empty:
    print("\nDuplicate author-year pairs in df1:")
    print(duplicate_pairs)


Total pairs: 78
Unique pairs: 69
Duplicate pairs: 9

Duplicate author-year pairs in df1:
first_author_lastname  year
Chang                  2004    2
Cooper                 1988    2
Forbes                 1990    2
Hersh                  1993    2
Mehlisch               2010    2
Moore                  1998    2
Searle                 2020    2
Seymour                1991    2
Yue                    2013    2
dtype: int64


In [ ]:
import json
import csv
import os

def json_to_csv(json_path):
    # Read JSON file
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # Flatten JSON records (only top-level + nested keys)
    flat_data = []
    for item in data.get("extracted_data", []):
        flat_item = {}
        for key, value in item.items():
            if isinstance(value, dict):
                # Flatten nested dictionaries
                for sub_key, sub_val in value.items():
                    flat_item[f"{key}.{sub_key}"] = sub_val
            else:
                flat_item[key] = value
        flat_data.append(flat_item)

    # CSV output path in same folder as JSON
    folder = os.path.dirname(json_path)
    base_name = os.path.splitext(os.path.basename(json_path))[0]
    csv_path = os.path.join(folder, f"{base_name}.csv")

    # Write to CSV
    with open(csv_path, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=flat_data[0].keys())
        writer.writeheader()
        writer.writerows(flat_data)

    print(f"CSV saved at: {csv_path}")


# Example usage:
# Provide the full path to your JSON file here
json_to_csv(r"/nlp/data/karthik9/Sprint1/Dental/langextract/test_results_gpt4o/extracted_result.json")


In [1]:
import re
import json
from typing import Dict, Any, List, Optional, Tuple
from dataclasses import dataclass

@dataclass
class ExtractedTable:
    """Represents a single extracted table with metadata"""
    table_id: str
    title: Optional[str]
    caption: Optional[str]
    headers: List[str]
    rows: List[List[str]]
    column_count: int
    row_count: int
    has_numeric_data: bool
    numeric_columns: List[int]  # indices of columns containing numbers
    metadata: Dict[str, Any]

class MarkdownTableExtractor:
    """Extract and reformat tables from markdown text"""
    
    def __init__(self):
        # More reliable markdown table regex
        self.table_pattern = re.compile(
            r'(\|.+\|\s*\n\|[-:\s|]+\|\s*(?:\n\|.+\|\s*)+)',
            re.MULTILINE
        )
        
    def extract_all_tables(self, markdown_text: str) -> List[ExtractedTable]:
        """Extract all tables from markdown text"""
        tables = []
        matches = self.table_pattern.finditer(markdown_text)
        
        for idx, match in enumerate(matches):
            table_text = match.group(0)
            table_start = match.start()
            
            # Extract potential title/caption before table
            title, caption = self._extract_table_context(markdown_text, table_start)
            
            extracted = self._parse_table(
                table_text=table_text,
                table_id=f"table_{idx + 1}",
                title=title,
                caption=caption
            )
            
            if extracted:
                tables.append(extracted)
        
        return tables
    
    def _extract_table_context(self, markdown: str, table_start: int) -> Tuple[Optional[str], Optional[str]]:
        """Extract title and caption information around the table"""
        context_start = max(0, table_start - 200)
        context = markdown[context_start:table_start].strip()
        lines = context.split('\n')
        
        title = None
        caption = None
        for line in reversed(lines[-3:]):  # check last 3 lines above table
            if line.strip().startswith("Table "):
                title = line.strip()
            elif not caption and len(line.strip()) > 10:
                caption = line.strip()
        
        return title, caption
    
    def _parse_table(self, table_text: str, table_id: str, 
                    title: Optional[str], caption: Optional[str]) -> Optional[ExtractedTable]:
        """Parse a markdown table into structured format"""
        lines = [l.strip() for l in table_text.strip().split('\n') if l.strip()]
        if len(lines) < 2:
            return None
        
        headers = self._parse_row(lines[0])
        if not headers:
            return None
        
        # Skip separator row
        data_start = 2 if len(lines) > 1 and re.match(r'^[\|\s\-:]+$', lines[1]) else 1
        
        rows = [self._parse_row(line) for line in lines[data_start:]]
        rows = [row for row in rows if row]  # remove empty rows
        
        if not rows:
            return None
        
        numeric_columns = self._identify_numeric_columns(rows)
        has_numeric = len(numeric_columns) > 0
        
        rows = self._clean_table_values(rows)
        headers = self._clean_header_values(headers)
        
        return ExtractedTable(
            table_id=table_id,
            title=title,
            caption=caption,
            headers=headers,
            rows=rows,
            column_count=len(headers),
            row_count=len(rows),
            has_numeric_data=has_numeric,
            numeric_columns=numeric_columns,
            metadata={'source': 'markdown', 'extraction_method': 'regex_parsing'}
        )
    
    def _parse_row(self, row_text: str) -> List[str]:
        row_text = row_text.strip().strip('|')
        return [c.strip() for c in row_text.split('|')]
    
    def _identify_numeric_columns(self, rows: List[List[str]]) -> List[int]:
        numeric_cols = []
        num_cols = len(rows[0]) if rows else 0
        for col_idx in range(num_cols):
            vals = [r[col_idx] for r in rows if col_idx < len(r) and r[col_idx].strip()]
            if not vals:
                continue
            numeric_like = [v for v in vals if self._is_numeric_value(v)]
            if len(numeric_like) / len(vals) > 0.6:
                numeric_cols.append(col_idx)
        return numeric_cols
    
    def _is_numeric_value(self, value: str) -> bool:
        cleaned = value.replace(',', '').replace('%', '').replace('$', '')
        cleaned = cleaned.replace('±', ' ').replace('(', '').replace(')', '')
        return bool(re.match(r'^-?\d+(\.\d+)?$', cleaned.strip()))
    
    def _clean_table_values(self, rows: List[List[str]]) -> List[List[str]]:
        return [[' '.join(c.replace('**', '').split()) for c in row] for row in rows]
    
    def _clean_header_values(self, headers: List[str]) -> List[str]:
        return [' '.join(h.replace('**', '').split()).rstrip(':') for h in headers]
    
    def tables_to_json(self, tables: List[ExtractedTable]) -> Dict[str, Any]:
        return {
            'total_tables': len(tables),
            'tables': [
                {
                    'id': t.table_id,
                    'title': t.title,
                    'caption': t.caption,
                    'headers': t.headers,
                    'rows': t.rows,
                    'structure': {
                        'columns': t.column_count,
                        'rows': t.row_count,
                        'has_numeric_data': t.has_numeric_data,
                        'numeric_column_indices': t.numeric_columns
                    }
                } for t in tables
            ]
        }

def integrate_table_extraction(markdown_text: str) -> Dict[str, Any]:
    extractor = MarkdownTableExtractor()
    tables = extractor.extract_all_tables(markdown_text)
    result = extractor.tables_to_json(tables)
    result['summary'] = {
        'total_tables_found': len(tables),
        'tables_with_numeric_data': sum(1 for t in tables if t.has_numeric_data)
    }
    return result

# Example usage:
if __name__ == "__main__":
    md = """![](_page_0_Picture_0.jpeg)\n\n![](_page_0_Figure_1.jpeg)\n\n**Current Medical Research and Opinion**\n\n**ISSN: 0300-7995 (Print) 1473-4877 (Online) Journal homepage: [www.tandfonline.com/journals/icmo20](https://www.tandfonline.com/journals/icmo20?src=pdf)**\n\n# **Longer analgesic effect with naproxen sodium than ibuprofen in post-surgical dental pain: a randomized, double-blind, placebo-controlled, single-dose trial**\n\n**Stephen A. Cooper, Paul Desjardins, Patrick Brain, Alberto Paredes-Diaz, Emanuel Troullos, Robert Centofanti & Bob An**\n\n**To cite this article:** Stephen A. Cooper, Paul Desjardins, Patrick Brain, Alberto Paredes-Diaz, Emanuel Troullos, Robert Centofanti & Bob An (2019) Longer analgesic effect with naproxen sodium than ibuprofen in post-surgical dental pain: a randomized, double-blind, placebocontrolled, single-dose trial, Current Medical Research and Opinion, 35:12, 2149-2158, DOI: [10.1080/03007995.2019.1655257](https://www.tandfonline.com/action/showCitFormats?doi=10.1080/03007995.2019.1655257)\n\n**To link to this article:** <https://doi.org/10.1080/03007995.2019.1655257>\n\n| View supplementary material             | Published online: 27 Aug 2019. |\n|-----------------------------------------|--------------------------------|\n| Submit your article to this journal     | Article views: 783             |\n| View related articles                   | View Crossmark data            |\n| Citing articles: 7 View citing articles |                                |\n\n#### ORIGINAL ARTICLE\n\n# Longer analgesic effect with naproxen sodium than ibuprofen in post-surgical dental pain: a randomized, double-blind, placebo-controlled, single-dose trial\n\nStephen A. Coopera , Paul Desjardinsb , Patrick Brain<sup>c</sup> , Alberto Paredes-Diazd , Emanuel Troullosd , Robert Centofantie and Bob Anf\n\na Stephen A. Cooper, DMD, PhD, LLC, Palm Beach Gardens, FL, USA; <sup>b</sup> Desjardins Associates, LLC, Maplewood, NJ, USA; <sup>c</sup> Jean Brown Research, Salt Lake City, UT, USA; <sup>d</sup> Global Medical Affairs, Bayer Consumer Health, Whippany, NJ, USA; <sup>e</sup> Clinical Development, Bayer Consumer Health, Whippany, NJ, USA; <sup>f</sup> Biostatistics, Bayer Consumer Health, Whippany, NJ, USA;\n\n#### ABSTRACT\n\nBackground: Non-steroidal anti-inflammatory drugs (NSAIDs) are recommended as first-line medications in mild-to-moderate acute pain. However, comparative data regarding the duration of analgesia for commonly-used NSAIDs at non-prescription doses is lacking. This study evaluated the time to rescue medication following a single dose of naproxen sodium (NAPSO) vs ibuprofen (IBU) and placebo in subjects with moderate-to-severe post-surgical dental pain.\n\nMethods: This single-center, randomized, double-blind, parallel group, placebo-controlled study included healthy subjects with moderate-to-severe baseline pain (Categorical Pain Intensity Scale) who also rated their pain 5 on a 0–10 pain intensity Numerical Rating Scale following extraction of two impacted mandibular third molars. A single oral dose of NAPSO (440 mg), IBU (400 mg), or placebo was administered. The primary efficacy endpoint was the time to first rescue medication, while secondary endpoints included the sum of pain intensity difference (SPID) and total pain relief (TOTPAR) over 24 h. ClinicalTrials.gov trial registration number: NCT03404206 (EudraCT 2017-005049-67).\n\nResults: In the per protocol population (n ¼ 385; mean age ¼ 19 years), the time to rescue medication was significantly (p < .001) longer with NAPSO than IBU and placebo. After treatment, the greatest separation of NAPSO from IBU occurred at 9–14 h and from placebo at 1–6 h. Fewer NAPSO subjects required rescue medication (58/166, 34.9%) compared with IBU (137/165, 83.0%) and placebo (44/54, 81.5%). SPID 0–24 h and TOTPAR 0–24 h were both greater with NAPSO than IBU or placebo.\n\nConclusions: The duration of pain relief after a single dose of NAPSO was significantly longer than after IBU, and significantly fewer NAPSO-treated subjects required rescue medication over a 24-h period.\n\n#### ARTICLE HISTORY\n\nReceived 16 July 2019 Revised 7 August 2019 Accepted 9 August 2019\n\n#### KEYWORDS\n\nAnalgesia; ibuprofen; naproxen sodium; pain; randomized controlled trial\n\n# Introduction\n\nNon-steroidal anti-inflammatory drugs (NSAIDs) are recommended as first-line drugs in many guidelines for a variety of acute pain episodes[1](#page-10-0)–<sup>5</sup> . Naproxen sodium (NAPSO) and ibuprofen (IBU) are among the most commonly used NSAIDs because they lack opioid side-effects, are markedly efficacious, and have a favorable safety profile6–[10,](#page-10-0) complemented by their availability as non-prescription drugs. However, there are few comparative studies regarding their relative analgesic efficacy at optimal non-prescription doses. Fast and effective pain relief is important to patients, and long-lasting relief is advantageous to enhance compliance. An NSAID with a long elimination half-life, such as NAPSO (10–15 h[11\\)](#page-10-0), has a long duration of actio[n12](#page-10-0) and requires only once or twice daily dosing. In contrast, an NSAID with a shorter half-life, such as IBU (half-life 1–2 h[11](#page-10-0)), has a relatively shorter duration of action that often requires more frequent dosing over a 24-h period.\n\nThe dental impaction pain model is well validated and has been used in hundreds of successful analgesic efficacy studies[13](#page-10-0). The model has excellent assay sensitivity because surgical removal of impacted third molars can be easily standardized and the target population is generally young and healthy. Most importantly, the post-surgical pain is often sustained, lasting for 24 h or longer. The sensitivity of this pain model allows comparison of several measures of analgesic efficacy, including reliable estimates of onset of analgesia, peak effects, and the duration of analgesic activity of various pharmacologic intervention[s13](#page-10-0)–15. Acute pain studies generally assess the duration of analgesia by measuring the time to re-medication[16](#page-10-0) with a rescue drug or a second dose of the same study drug.\n\nThe difference in pharmacokinetic profiles between NAPSO and IBU suggests that NAPSO should show a longer time to remedication, but direct evidence has been lacking. In two previous single-dose dental pain studies comparing NAPSO to IB[U17,18,](#page-10-0) the duration of analgesia of NAPSO was longer than IBU, but the difference in those studies did not reach statistical significance. However, both were 12-h outpatient studies, which may have been too short of an\n\nCONTACT Alberto Paredes-Diaz alberto.paredes-diaz@bayer.com Bayer Consumer Health, 100 Bayer Blvd, Whippany, NJ 07981, USA Supplemental data for this article is available online at [https://doi.org/10.1080/03007995.2019.1655257.](https://doi.org/10.1080/03007995.2019.1655257)\n\nwww.cmrojournal.com\n\n2019 Informa UK Limited, trading as Taylor & Francis Group\n\nevaluation period to fully assess their duration of analgesia. Furthermore, those studies appear to have been under-powered to detect a difference in duration of effective analgesia.\n\nThe primary purpose of our study was to use the dental impaction pain model to compare the duration of action of a single oral, non-prescription dose of NAPSO (440 mg) to IBU (400 mg) and placebo over 24 h. Secondary objectives were to compare the overall analgesic effects (Summed Pain Intensity Difference [SPID] and Total Pain Relief [TOTPAR]) over 24 h post-dosing. The study was specifically designed to assess the acute post-surgical dental pain episode in subjects who could benefit from the administration of a non-prescription, non-opioid pain reliever.\n\n## Methods\n\n#### Study design\n\nThis was a single-center, randomized, double-blind, placebocontrolled study in healthy subjects experiencing moderateto-severe post-operative dental pain following the surgical removal of two mandibular impacted third molars, and who also rated their pain intensity 5 on a 0–10 Numerical Rating Scale (NRS). We evaluated the analgesic efficacy and safety of a single dose of NAPSO (440 mg) or IBU (400 mg) and placebo over 24 h. The trial was conducted between February 12, 2018 and July 10, 2018. The study protocol was approved by IntegReview IRB (Austin, TX). The endpoints and study design were consistent with FDA Guidance and the methods advocated by expert[s13,19.](#page-10-0) The study was conducted in accordance with the Declaration of Helsinki and in compliance with all current Good Clinical Practice guidelines. The ClinicalTrials.gov trial registration number is NCT03404206 (EudraCT 2017-005049-67).\n\n#### Subject population\n\nSubjects who sought elective third molar surgery were solicited to participate. All were healthy, ambulatory, male and non-pregnant female volunteers between 16–40 years of age, with a body mass index between 18.5–30.0 kg/m<sup>2</sup> inclusive. Individuals with a history of hypersensitivity to any NSAID, history of clinically significant disease or malignancies, current history of gastrointestinal illness, active dental infection, relevant concomitant disease such as asthma (exercise-induced asthma was permitted), positive urine drug screen, or habituation to analgesics were not eligible to participate. All subjects (or parent/guardian for subjects under 18 years old) signed the IRB-approved written informed consent. Subjects under 18 years of age signed an IRB-approved written assent form.\n\nSubjects were scheduled to undergo surgical removal of at least two mandibular third molars that were partially or fully bony impacted under local anesthesia ± light sedation with nitrous oxide/oxygen. Mandibular molars had to demonstrate modified Demirjian root classification stage D, E, F, G, or [H20.](#page-10-0) In addition, the two maxillary third molars could be removed, regardless of the type of impaction. Any supernumerary molars could be removed at the discretion of the oral surgeon. Patients fasted from midnight prior to surgery until completion of surgery, and continued fasting (apart from clear liquids) until after study drug administration. Subjects were screened prior to surgery: urine for drug abuse and cotinine, and a breathalyzer for alcohol use.\n\nSubjects who had moderate-to-severe post-operative pain (baseline Categorical Pain Intensity Scale, where 0 ¼ no pain, 1 ¼ mild pain, 2 ¼ moderate pain, 3 ¼ severe pain) and a pain intensity NRS score (0 ¼ no pain to 10 ¼ worst possible pain) of 5 within 4.5 h post-surgery (but no later than 14:45 h) continued in the study and were randomized to study treatment. Subjects not meeting these criteria were withdrawn from the study.\n\n#### Treatments administered\n\nA single dose of one of the three study treatments, as determined by a pre-determined randomization schedule, was administered: oral NAPSO 220 mg two tablets (Aleve Caplets, Bayer Healthcare LLC, Whippany, NJ, USA; total dose 440 mg), oral IBU 200 mg two tablets (Advil Caplets, Pfizer Consumer Healthcare, Madison, NJ, USA; total dose 400 mg), or placebo two tablets, all taken with a 8 oz glass of water. Subjects who did not achieve sufficient pain relief could request rescue medication at any time during the study, although they were encouraged to wait 90 min, if possible, to allow the investigational product to take effect. Rescue medication was acetaminophen 325 mg plus hydrocodone 5 mg, or other appropriate analgesics approved for treating acute pain, according to the discretion of the investigator. Following discharge from the research facility, analgesic medication was up to the discretion of the investigator according to standard practice. No other medication, apart from oral contraceptives, prophylactic antibiotics, multivitamin supplements, or other routine medications to treat benign conditions such as acne, was allowed throughout the study.\n\n#### Randomization and blinding\n\nA unique subject identifier number was assigned in numerical order stratified by Baseline Pain (moderate or severe) on the categorical pain scale. Stratification was used in this study to ensure that the subject's perceived pain severity (moderate or severe) was balanced across the treatment groups prior to administration of study medication. The two active study treatments and placebo were administered in a 3:3:1 randomization ratio (NAPSO: IBU: placebo) based on a computer-generated randomization schedule.\n\nAll subjects, investigators, and staff involved in pain assessment were blinded to the treatments administered, and subjects were blindfolded during drug administration to preserve blinding. The study drug was dispensed by an unblinded study team member based on the randomization schedule; that team member had no other role in the study and did not reveal the identity of the study drug to any members of the blinded study team.\n\n<span id=\"page-3-0\"></span>![](_page_3_Figure_1.jpeg)\n\nFigure 1. Study design. Pain relief assessed using the Categorical Pain Relief Rating Scale (0 <sup>¼</sup> no relief, 1 <sup>¼</sup> a little relief, 2 <sup>¼</sup> some relief, 3 <sup>¼</sup> a lot of relief, 4 ¼ complete relief). Pain intensity and pain relief were measured at 0.5, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 16, 18, 20, 22, and 24 h post-dose. Abbreviation. NRS, numerical rating scale (0 ¼ no pain to 10 ¼ worst possible pain).\n\n#### Clinical assessments\n\nAssessments were performed at scheduled intervals over the 24-h period post-dosing (Figure 1 and [Supplementary](https://doi.org/10.1080/03007995.2019.1655257) [Table 1\\)](https://doi.org/10.1080/03007995.2019.1655257). Pain intensity was assessed by subjects using the 0–10 NRS scale, pain relief was evaluated using the Categorical Pain Relief Rating Scale (0 ¼ no relief, 1 ¼ a little relief, 2 ¼ some relief, 3 ¼ a lot of relief, and 4 ¼ complete relief), Whether the Baseline Pain was Half Gone was assessed as No (0) or Yes (1), while the Global Assessment of the investigational product as a pain reliever was graded on a 5-point scale (0 ¼ poor, 1 ¼ fair, 2 ¼ good, 3 ¼ very good, 4 ¼ excellent) at 24 h or immediately prior to the first dose of rescue medication during the 24-h evaluation period, whichever came first. Safety was assessed by spontaneous reports and observations of adverse events (AEs) using non-leading questions and any notable changes in vital signs.\n\n#### Efficacy endpoints\n\nFor each post-dose time point, a Pain Intensity Difference (PID) was derived by subtracting the pain intensity at the post-dose time point from the Baseline Pain Score (NRS Baseline Score – NRS post-Baseline Score); a positive difference was indicative of improvement. Time-weighted SPIDs were calculated by multiplying the PID score at each postdose time point by the duration (in hours) since the preceding time point and then summing these values over the desired time interval. Similarly, the Total Pain Relief Scores (TOTPARs) were calculated by multiplying the Pain Relief Score at each post-dose time point by the duration (in hours) since the preceding time point and then summing these values.\n\nThe primary efficacy endpoint was the time to first use of rescue medication. Secondary endpoints were the overall analgesic effects (SPID 0–24) and overall relief from starting pain (TOTPAR 0–24). Other endpoints included SPID and TOTPAR over hours 0–4, 0–8, 0–12, and 6–12, Peak Pain Intensity and Peak Pain Relief, Sum of Observations with Pain Half Gone, the duration of pain relief at least half gone, the proportion of subjects reporting their pain as half gone or better at each time point, and the total amount of time that a subject reported at least a 2-point PID improvement.\n\n#### Statistical analysis\n\nAssuming that 22.2% of NAPSO subjects and 35.8% of IBU subjects would not need the rescue medication by 12 h following the study drug administation, as observed in a prior study[18](#page-10-0), a total of 364 subjects (156 subjects per active treatment arm and 52 in the placebo using a 3:3:1 ratio) were required to achieve 80% of power with a type I error of 0.05. Thus, at least 385 subjects were to be randomized into the study assuming a drop-out rate of 5%.\n\nAll analyses were performed using SAS Version 9.[31](#page-9-0) or higher unless otherwise stated. Except where noted, all statistical tests were two-sided and performed at the 0.05 level of significance. No adjustments for multiplicity were made. Descriptive statistics were used to provide an overview of the efficacy and safety results. For categorical parameters, the number and percentage of subjects in each category were presented. For continuous parameters, descriptive statistics included N (number of subjects [sample size]), n (number of observations), mean, standard deviation (SD), median, minimum, and maximum. The primary efficacy analysis was performed on the per protocol (PP) population (all subjects who provided at least one pain assessment after the first dose of the investigational product, who did not have any major protocol violations and were analyzed according to the treatment they actually received) and corroborated using data from the intent-to-treat (ITT) population. The PP data are presented here, as the populations for PP (n ¼ 385) and ITT (n ¼ 387) were virtually identical. The safety analysis was conducted on all subjects who were randomized and took the one dose of investigational product.\n\n#### Primary efficacy analysis\n\nTime (h) to first use of rescue medication during the treatment period was the primary efficacy endpoint. If a subject did not take the rescue medication during the treatment period, the subject was considered censored at the time of last assessment (24 h post-dose). Time to first use of rescue medication was estimated and plotted using Kaplan-Meier method and analyzed using a log-rank test stratified by baseline pain intensity. Kaplan-Meier 25th, 50th, and 75th percentiles, as well as minimum and maximum times observed, were determined. Log-rank tests were used to compare NAPSO to IBU, and NAPSO to placebo.\n\n#### Secondary and other efficacy analyses\n\nSPID 0–24 and TOTPAR 0–24, as well as SPID and TOTPAR at 0–4, 0–8, 0–12, and 6–12 h, were analyzed using an analysis of covariance model (ANCOVA), with treatment as fixed effect and baseline pain intensity score as the covariate. The least squares mean (LSMean) ± SD and 95% confidence intervals (CIs) for the treatment differences (NAPSO-placebo; NAPSO-IBU; and IBU-placebo) were also calculated. The cumulative proportion of subjects taking rescue medication by each time point was summarized and compared using Chi-square tests. The global assessment score was compared using the Chi-square test with modified ridit score.\n\n#### Adverse events\n\nAll AEs were recorded and classified on the basis of MedDRA terminology. Treatment-emergent AEs (TEAEs) were defined as AEs with an onset on or after the date of the first study drug administration. AEs noted prior to the first study drug administration that worsened after baseline were also reported.\n\n#### Missing data\n\nThe last observation carried forward (LOCF) method was used to extrapolate efficacy data that were missing beyond the last observation. Missing scheduled pain and relief scores prior to the last observation were imputed using the weighted average of adjacent evaluations. Missing AE start dates were imputed using partial date imputation rules. In all analyses, all pain intensity scores after intake of rescue medication were replaced by the baseline pain score or the score assessed immediately before taking rescue medication, whichever was worse. All pain relief scores after intake of rescue medication were replaced by \"no relief\" (0).\n\n### Results\n\n#### Subject population\n\nA total of 387 subjects were randomized to treatment, 166 subjects each to NAPSO or IBU and 55 subjects to placebo [\\(Figure 1](#page-3-0)). Four (2.4%) subjects withdrew from the NAPSO group and two (1.2%) from the IBU group (lost to follow-up or withdrawal by the subject). The PP population included 385 patients, as two subjects were excluded: one from the placebo group (use of rescue medication at or prior to 60 min after ingesting study medication) and one from the IBU group (use of interfering concomitant medication). The safety population included all 387 randomized subjects.\n\nDemographic characteristics were comparable across the treatment groups, although there was a slightly larger proportion of males in the NAPSO group ([Table 1](#page-5-0)). Overall, the mean age was 19.0 (range ¼ 16–35) years and the mean number of teeth extracted was 3.8; all subjects had extractions of the left and right mandibular third molars and a slightly lower percentage (86.7–89.8%) had extraction of the left and right maxillary third molars. The mean (range) dental surgery duration from first incision to last suture was 13.3 (3–54) min. The mean categorical pain intensity score at baseline was 2.6; severe pain was reported in 59.0% (228/387) of subjects and moderate pain in 41.0% (159/387) of subjects. The mean NRS score at baseline was 7.5 in all three groups.\n\n#### Primary endpoint\n\nAnalysis of the primary efficacy endpoint, time to first use of rescue medication in the PP population, demonstrated a statistically significant and clinically relevant superiority of NAPSO over IBU and placebo (p < .001) ([Table 2](#page-5-0) and [Figure 2\\)](#page-6-0). Fewer than 50% of subjects in the NAPSO group needed rescue medication during the 24-h follow-up period; in contrast, the time taken before 50% of subjects had used rescue medication (i.e., the 50th percentile) was 10.5 h in the IBU group and 2.5 h in the placebo group ([Table 2](#page-5-0)). The NAPSO group displayed a clear and significant separation from the IBU group from 9 h after treatment (p < .05) through to the end of the 24-h period ([Figure 2](#page-6-0)). The proportion of patients who did not use rescue medication and were censored at the time of last assessment ([Table 2\\)](#page-5-0) was significantly higher in the NAPSO group (108/166, 65.1%) compared with placebo (10/54, 18.5%) and IBU (28/165, 17.0%) (p < .001, respectively). At 6 h after treatment, the proportion of subjects taking rescue medication rose noticeably in the IBU group, but only slightly in the NAPSO group [\\(Figure 2](#page-6-0)). The same results were observed for the ITT population.\n\n#### Secondary endpoints\n\nThe reduction in pain intensity over 24 h (SPID0-24) was significantly greater with NAPSO (LSMean 83.30) than with\n\n|  |  | Table 1. Subject demographics and characteristics (per protocol population). |  |  |  |  |  |\n|--|--|------------------------------------------------------------------------------|--|--|--|--|--|\n|--|--|------------------------------------------------------------------------------|--|--|--|--|--|\n\n<span id=\"page-5-0\"></span>\n\n| Table 1. Subject demographics and characteristics (per protocol population).        | Placebo (n ¼ 54)                                    | Ibuprofen (400 mg) (n ¼ 165)                        | Naproxen sodium<br>(440 mg) (n ¼ 166)                | Overall (n ¼ 385)                                    |\n|-------------------------------------------------------------------------------------|-----------------------------------------------------|-----------------------------------------------------|------------------------------------------------------|------------------------------------------------------|\n| Age (years), mean ± SD (range)                                                      | 19.0 ± 2.93 (16–27)                                 | 19.0 ± 2.61 (16–35)                                 | 19.0 ± 2.96 (16–32)                                  | 19.0 ± 2.80 (16–35)                                  |\n| Sex, n (%)                                                                          |                                                     |                                                     |                                                      |                                                      |\n| Male                                                                                | 25 (46.3)                                           | 77 (46.7)                                           | 89 (53.6)                                            | 191 (49.6)                                           |\n| Female                                                                              | 29 (53.7)                                           | 88 (53.3)                                           | 77 (46.4)                                            | 194 (50.4)                                           |\n| Weight (kg), mean ± SD (range)<br>Body mass index (kg/m2<br>),<br>mean ± SD (range) | 67.2 ± 10.51 (43.5–92.5)<br>23.5 ± 2.94 (18.7–29.7) | 68.1 ± 10.61 (46.3–95.9)<br>23.8 ± 2.77 (18.5–29.9) | 68.0 ± 11.61 (45.4–112.9)<br>23.6 ± 2.91 (18.5–29.9) | 67.9 ± 11.01 (43.5–112.9)<br>23.7 ± 2.85 (18.5–29.9) |\n| Ethnicity, n (%)                                                                    |                                                     |                                                     |                                                      |                                                      |\n| Hispanic or Latino                                                                  | 9 (16.7)                                            | 25 (15.2)                                           | 23 (13.9)                                            | 57 (14.8)                                            |\n| Not Hispanic or Latino                                                              | 45 (83.3)                                           | 140 (84.8)                                          | 143 (86.1)                                           | 328 (85.2)                                           |\n| Race                                                                                |                                                     |                                                     |                                                      |                                                      |\n| White                                                                               | 47 (87.0)                                           | 147 (89.1)                                          | 148 (89.2)                                           | 342 (88.8)                                           |\n| Black or African American                                                           | 1 (1.9)                                             | 4 (2.4)                                             | 2 (1.2)                                              | 7 (1.8)                                              |\n| American Indian or<br>Alaska Native                                                 | 2 (3.7)                                             | 4 (2.4)                                             | 3 (1.8)                                              | 9 (2.3)                                              |\n| Asian                                                                               | 1 (1.9)                                             | 2 (1.2)                                             | 1 (0.6)                                              | 4 (1.0)                                              |\n| Native Hawaiian or other<br>Pacific Islander                                        | 1 (1.9)                                             | 3 (1.8)                                             | 1 (0.6)                                              | 5 (1.3)                                              |\n| Other                                                                               | 2 (3.7)                                             | 5 (3.0)                                             | 11 (6.6)                                             | 18 (4.7)                                             |\n| Number of teeth extracted,<br>mean ± SD (range)                                     | 3.8 ± 0.60 (2–4)                                    | 3.8 ± 0.64 (2–6a<br>)                               | 3.8 ± 0.54 (2–4)                                     | 3.8 ± 0.59 (2–6)                                     |\n| Duration of surgery (min),<br>mean ± SD (range)                                     | 13.9 ± 7.34 (5–35)                                  | 13.6 ± 6.48 (3–50)                                  | 12.8 ± 7.27 (4–54)                                   | 13.3 ± 6.95 (3–54)                                   |\n| Baseline pain intensity score,<br>n (%)                                             |                                                     |                                                     |                                                      |                                                      |\n| Moderate pain (2)                                                                   | 23 (42.6)                                           | 66 (40.0)                                           | 69 (41.6)                                            | 158 (41.0)                                           |\n| Severe pain (3)                                                                     | 31 (57.4)                                           | 99 (60.0)                                           | 97 (58.4)                                            | 227 (59.0)                                           |\n| Mean ± SD (range)                                                                   | 2.6 ± 0.50 (2–3)                                    | 2.6 ± 0.49 (2–3)                                    | 2.6 ± 0.49 (2–3)                                     | 2.6 ± 0.49 (2–3)                                     |\n| Baseline NRS score,<br>mean ± SD (range)                                            | 7.5 ± 1.27 (5–10)                                   | 7.5 ± 1.10 (5–10)                                   | 7.5 ± 1.04 (5–10)                                    | —                                                    |\n\na One subject had four third molars and two supernumerary teeth extracted.\n\nAbbreviations. NRS, numerical rating scale; SD, standard deviation.\n\nTable 2. Primary efficacy endpoint: Kaplan-Meier estimates for time to rescue medication (per protocol population).\n\n|                                                                                                 | Placebo (n ¼ 54) | Ibuprofen (n ¼ 165) | Naproxen sodium (n ¼ 166) |\n|-------------------------------------------------------------------------------------------------|------------------|---------------------|---------------------------|\n| Number of subjects who took rescue medication, n (%)                                            | 44 (81.5)        | 137 (83.0)          | 58 (34.9)                 |\n| Number of subjects censored (i.e. no rescue medication was<br>required during the study), n (%) | 10 (18.5)        | 28 (17.0)           | 108 (65.1)                |\n| Time to achieving event (h)                                                                     |                  |                     |                           |\n| Minimuma                                                                                        | 1.18             | 1.22                | 1.22                      |\n| 25th percentileb                                                                                | 2.12             | 8.27                | 11.02                     |\n| 50th percentileb                                                                                | 2.53             | 10.53               | NE                        |\n| 75th percentileb                                                                                | 12.10            | 14.12               | NE                        |\n| Maximuma                                                                                        | 21.75            | 18.28               | 22.17                     |\n| p-valuec                                                                                        | <.001            | <.001               |                           |\n\na Observed time to event;\n\nb estimated using Kaplan-Meier methods;\n\nc p-values are from log-rank tests stratified by baseline pain intensity comparing the treatment to naproxen sodium.\n\nNote: Percentages are based on the number of subjects in the per protocol population for each treatment group.\n\nAbbreviation. NE, not estimable.\n\neither IBU (48.57) or placebo (9.87) (p < .05, respectively; [Table 3\\)](#page-6-0). Total Pain Relief over 24 h (TOTPAR0-24) was also significantly greater with NAPSO (LSMean 47.16) vs IBU (28.95) or placebo (13.44) (p < .05, respectively; [Table 3\\)](#page-6-0). The treatment differences compared with placebo for SPID0-24 and TOTPAR0-24 were approximately twice as large with NAPSO (73.43 and 33.72, respectively) compared with IBU (34.73 and 18.21, respectively).\n\n#### Other efficacy variables\n\nThe time-weighted summaries of the SPID and TOTPAR scores were significantly greater (p < 0.05, respectively, based on the 95% CI) with NAPSO than IBU over the intervals 0–12 and 6–12 h [\\(Table 3\\)](#page-6-0). Comparable time-weighted summaries of the SPID and TOTPAR scores were observed over the intervals 0–4 and 0–8 h.\n\nIndividual PID and Pain Relief scores by time point [\\(Figure 3](#page-7-0)) revealed a rapid onset of analgesic effect that was similar between the two active treatments until 7 h. Beginning at 6 h, the analgesic effect started to decline in the IBU group and was essentially comparable with placebo by 14 h. In contrast, the analgesic effect of the NAPSO group was maintained throughout the 24-h period [\\(Figure 3\\)](#page-7-0). The mean Peak PID and Pain Relief scores (LS means ± SD) were similar between NAPSO (5.6 ± 1.93 and 3.0 ± 0.90, respectively) and IBU (5.7 ± 1.93 and 3.1 ± 0.90), and were more than twice as high as those obtained with placebo (2.0 ± 1.93 and 1.2 ± 0.90). However, the median total time of subjects reporting at least a 2-point improvement for PID was\n![](0__page_6_Figure_1.jpeg)\n\nFigure 2. Primary efficacy endpoint: Kaplan-Meier estimates for time to rescue medication (per protocol population). Treatments administered were either oral naproxen sodium (220 mg two tablets, total dose 440 mg; n ¼ 166), oral ibuprofen (200 mg two tablets, total dose 400 mg; n ¼ 165), or placebo (n ¼ 54).\n\n| Table 3. Secondary efficacy endpoints: SPIDs and TOTPAR scores at 24 h (per protocol population). | Placebo (n ¼ 54) | Ibuprofen (n ¼ 165) | Naproxen sodium (n ¼ 166) |\n|---------------------------------------------------------------------------------------------------|------------------|---------------------|---------------------------|\n| Secondary endpoints                                                                               |                  |                     |                           |\n| SPID 0–24                                                                                         |                  |                     |                           |\n| LS Mean ± SD                                                                                      | 9.87 ± 50.58     | 48.57 ± 50.58       | 83.30 ± 50.58             |\n| TOTPAR 0–24                                                                                       |                  |                     |                           |\n| LS Mean ± SD                                                                                      | 13.44 ± 24.59    | 28.95 ± 24.59       | 47.16 ± 24.59             |\n| Other endpoints                                                                                   |                  |                     |                           |\n| SPID (LS Mean ± SD)                                                                               |                  |                     |                           |\n| SPID 0–4                                                                                          | 2.03 ± 7.06      | 17.12 ± 7.06        | 16.31 ± 7.06              |\n| SPID 0–8                                                                                          | 4.01 ± 15.46     | 32.86 ± 15.46       | 32.39 ± 15.46             |\n| SPID 0–12                                                                                         | 5.57 ± 24.18     | 40.24 ± 24.18       | 46.08 ± 24.18             |\n| SPID 6–12                                                                                         | 2.42 ± 14.82     | 14.10 ± 14.82       | 21.31 ± 14.82             |\n| TOTPAR (LS Mean ± SD)                                                                             |                  |                     |                           |\n| TOTPAR 0–4                                                                                        | 2.48 ± 3.24      | 9.58 ± 3.24         | 9.10 ± 3.24               |\n| TOTPAR 0–8                                                                                        | 5.02 ± 7.18      | 18.40 ± 7.18        | 18.08 ± 7.18              |\n| TOTPAR 0–12                                                                                       | 7.23 ± 11.49     | 23.03 ± 11.49       | 25.91 ± 11.49             |\n| TOTPAR 6–12                                                                                       | 3.40 ± 7.33      | 8.44 ± 7.33         | 12.11 ± 7.33              |\n| Peak PID (LS Mean ± SD)                                                                           | 2.0 (1.93)       | 5.7 (1.93)          | 5.6 (1.93)                |\n| Peak Pain Relief (LS Mean ± SD)                                                                   | 1.2 (0.90)       | 3.1 (0.90)          | 3.0 (0.90)                |\n| Pain Half Gone                                                                                    |                  |                     |                           |\n| At least once in 24 h (%)                                                                         | 33.3             | 94.5                | 89.2                      |\n| Duration (hours) (LS Mean ± SD)                                                                   | 4.06 (7.68)      | 8.68 (7.68)         | 14.05 (7.68)              |\n| At Least a 2-Point PID                                                                            |                  |                     |                           |\n| Total time (hours) (LS Mean ± SD)                                                                 | 5.02 (7.53)      | 10.79 (7.53)        | 16.65 (7.53)              |\n| Global Assessment (Mean ± SD)                                                                     | 0.8 (1.15)       | 2.7 (0.94)#         | 2.7 (1.05)#               |\n\nLS Means are from an analysis of covariance with treatment as fixed effect and baseline pain intensity score as the covariate.\n\nStatistically significant at <sup>p</sup> <sup>&</sup>lt; .05 compared to both ibuprofen and placebo.\n\nStatistically significant at p < .05 compared to placebo.\n\n#Statistically significant at p < .001 compared to placebo based on a Chi-square test using the modified ridit score option.\n\nAbbreviations. CI, confidence interval; LS, least squares; SD, standard deviation; SPID, sum of pain intensity differences; TOTPAR, total pain relief.\n\nsubstantially longer for NAPSO (22.79 h than IBU (9.07 h) or placebo (0.00 h).\n\nOverall, the proportion of subjects reporting that their Baseline Pain was Half Gone at least once by 24 h post-dose was comparable for NAPSO (148/166, 89.2%) and IBU (156/165, 94.5%), both being much higher than for placebo (18/54, 33.3%), whereas the median total time of subjects reporting that their Baseline Pain was Half Gone was longer for NAPSO (17.64 h) than IBU (6.98 h) or placebo (0.00 h) (p < 0.05). The proportion of subjects reporting that their Baseline Pain was Half Gone at each observation over the 24-h period was similar for approximately the first 6 h postdose in the NAPSO (e.g., 34/166, 20.5% at 0.5 h; 120/166, 72.3% at 6 h) and IBU groups (51/165, 30.9% at 0.5 h; 115/165, 69.7% at 6 h), and both were greater than in the placebo group (1/54, 1.9% at 0.5 h; 11/54, 20.4% at 6 h) [\\(Figure 4\\)](#page-8-0). After 6 h, the proportion of subjects reporting that their Baseline Pain was Half Gone substantially decreased in the IBU group (range ¼ 13.3–61.8%, 22/165–102/165), and was comparable with placebo from 14 h (14.5% vs 18.5%,\n\n![](0__page_7_Figure_1.jpeg)\n\nFigure 3. Mean pain intensity differences (a) and mean pain relief scores (b) over 24 h (per protocol population). Pain intensity differences (based on the Numerical Rating Scale, 0 ¼ no pain to 10 ¼ worst possible pain) were calculated as baseline score – post-baseline score. Total Pain Relief was assessed using the Categorical Pain Relief Rating Scale (0 ¼ no relief, 1 ¼ a little relief, 2 ¼ some relief, 3 ¼ a lot of relief, and 4 ¼ complete relief). Treatments administered were either oral naproxen sodium (220 mg two tablets, total dose 440 mg; n ¼ 166), oral ibuprofen (200 mg two tablets, total dose 400 mg; n ¼ 165), or placebo (n ¼ 54).\n\n24/165 vs 10/54, respectively) onwards, while only a small decrease was observed for NAPSO (range ¼ 51.2–66.9%; 85/166–111/166) from 7–24 h post-dose ([Figure 4\\)](#page-8-0).\n\nA smaller proportion of subjects from the NAPSO (range ¼ 1.8–19.3%; 3/166–32/166) and IBU (range ¼ 1.8–20.0%, 3/165–33/165) groups required rescue medication from 2–8 h post-dose compared with the placebo group (range ¼ 20.4–70.4%; 11/54–38/54) ([Figure 5\\)](#page-8-0). However, the cumulative proportion of subjects taking rescue medication rose noticeably from 6 h in the IBU group. From 9 h through 24 h, the hourly cumulative percentages were lower in the NAPSO group (range ¼ 19.3–34.9%, 32/166–58/166) compared to the IBU group (range ¼ 30.3–83.0%, 50/165–137/165) ([Figure 5\\)](#page-8-0).\n\nThe percentage of subjects who rated their treatment as a good, very good, or excellent pain reliever was similar in the NAPSO (144/163; 88.3%) and IBU (147/165; 89.1%) groups, both higher than in the placebo group (14/53; 26.4%; most rated placebo as poor or fair). It should be noted that the timing of these global evaluations differed between subjects, as they were performed before taking the first dose of rescue medication.\n\n#### Safety\n\nAll three treatments were well tolerated, and the proportion of TEAEs in the active treatment groups was comparable. The proportion of subjects who reported at least one TEAE was lowest in the NAPSO group (26/166, 15.7%), followed by the IBU (31/166, 18.7%) and placebo (16/55, 29.1%) groups; no TEAE was considered related to the study drug [\\(Supplementary Table 2\\)](https://doi.org/10.1080/03007995.2019.1655257). The most frequently-reported\n\n<span id=\"page-8-0\"></span>![](0__page_8_Figure_1.jpeg)\n\nFigure 4. Proportion of subjects reporting pain half gone over 24 h (per protocol population). Response at each time point includes only subjects who have not taken a rescue medication at or prior to that time point. Treatments administered were either oral naproxen sodium (220 mg two tablets, total dose 440 mg; n ¼ 166), oral ibuprofen (200 mg two tablets, total dose 400 mg; n ¼ 165), or placebo (n ¼ 54).\n\n![](0__page_8_Figure_3.jpeg)\n\nFigure 5. Cumulative proportion of subjects taking rescue medication over 24 h (per protocol population). Treatments administered were either oral naproxen sodium (220 mg two tablets, total dose 440 mg; n ¼ 166), oral ibuprofen (200 mg two tablets, total dose 400 mg; n ¼ 165), or placebo (n ¼ 54).\n\nTEAEs were nausea, vomiting, and headache; these were most often reported after treatment with placebo (20.0%, 12.7%, and 12.7% (11/55, 7/55, and 7/55), respectively) and less often after treatment with NAPSO (5.4%, 0.6%, and 1.8% (9/166, 1/166, and 3/166), respectively) and IBU (10.2%, 4.2%, and 3.6% (17/166, 7/166, and 6/166) ([Supplementary](https://doi.org/10.1080/03007995.2019.1655257) [Table 3\\)](https://doi.org/10.1080/03007995.2019.1655257). In the NAPSO group, the only TEAE with a rate >2% above placebo was alveolar osteitis (2.4% vs 0%, 4/166 vs 0/166, respectively), which was not considered to be related to the study drug; there were no such TEAEs in the IBU group. One subject in the NAPSO group experienced a serious AE (severe appendicitis), which was not related to treatment or to protocol-required procedures. There were no deaths during the study, and no withdrawals were attributable to AEs.\n\n### Discussion\n\nOur randomized, double-blind, placebo-controlled trial in subjects experiencing moderate-to-severe dental impaction surgery pain has demonstrated the superiority of a maximum single, non-prescription dose of NAPSO over IBU and placebo for both overall efficacy and a significantly longer time to rescue medication (p < .001), with a clear difference between NAPSO and IBU after 9 h. The apparent onset and peak effect of NAPSO was equivalent to IBU, but NAPSO maintained a significantly longer duration of analgesia throughout the 24 h study period in a majority of subjects. This is consistent with the human pharmacokinetics of both drugs. In contrast, the mean pain intensity differences and mean pain relief scores in the IBU group began to decline after 6 h until both <span id=\"page-9-0\"></span>variables were comparable to placebo at 14 h and thereafter. Accordingly, the proportion of subjects who had sustained pain relief and did not require rescue medication during the 24-h study period was almost 4-times higher in the NAPSO group compared with the IBU and placebo groups (65.1% vs 17% and 18.5%, respectively). Two other efficacy variables from this study support the conclusion that naproxen's analgesic effect is substantially longer in duration than IBU. NAPSO-treated subjects experienced a longer period during which they reported their baseline pain \"Half Gone\". In addition, the NRS pain intensity differences showed \"at least a 2-point improvement\" for a substantially longer period in the NAPSO subjects compared to subjects treated with IBU or placebo. These outcome variables represent clinically meaningful degrees of pain reduction for extended periods with NAPSO compared to the other treatments.\n\nThese study results represent clinically meaningful benefits to patients and point to the value of having a long-acting, highly effective, and safe analgesic to treat acute pain medications. Our results are important for two reasons. First, because there is currently little head-to-head evidence regarding the duration of analgesic effect of commonly-used NSAIDs. Previous direct comparison of NAPSO and IBU in post-operative dental pain could only show a slightly longer duration of action with NAPSO that did not reach statistical significance[17,18](#page-10-0). Both were single-dose, randomized, parallel, single-center studies. One stud[y17](#page-10-0) compared naproxen sodium 220 mg, ibuprofen 200 mg, and placebo in subjects who had at least moderate pain after surgical removal of three or four third molars. The other stud[y18](#page-10-0) compared naproxen sodium 440 mg, ibuprofen 400 mg, and placebo in subjects who had at least moderate pain after surgical removal of one or two bony impacted third molars. However, both of these earlier studies had shorter (12 h) observation periods, and after 2 h the evaluations were performed by subjects at home using at-home paper diaries, which could have limited the ability to assess relative differences in later hours, e.g. subjects falling asleep. The design of our in-patient study is different in that subjects were under close observation by trained nursing staff throughout a 24-h period, including an overnight period where patients were awakened prior to each scheduled pain assessment. Second, our study has clearly demonstrated the substantial effectiveness of NSAIDs in treating an acute pain condition that is too often treated unnecessarily with opioid analgesics. Supporting our results, evidence from systematic reviews shows that NSAIDS offer a better balance of delivering powerful pain relief while minimizing acute adverse events compared to oral opioid analgesic[s21.](#page-10-0) Indeed, the American Dental Association recommends that dentists should consider NSAIDs as the first-line therapy for acute pain managemen[t2](#page-10-0) .\n\nThe study has some limitations. Our study was focused on the duration of effect of a single dose of medication for treating an acute episode of post-extraction dental pain. While the study is extremely useful for examining duration of effect, it is not intended to assess efficacy or tolerability after multiple doses of medication. Also, the adverse event assessment reported on the occurrence of events following administration of study medication regardless of rescue medication use. These rescue medications (hydrocodone/acetaminophen and tramadol) have their own associated side effects. The frequencies reported in our trial reflect the overall adverse event experience resulting from both the assigned study medication and any rescue medication provided to the subjects. Further, all pain intensity scores after intake of rescue medication were replaced by the baseline pain score or the score assessed immediately before taking rescue medication, whichever was worse, which could have affected the SPID results. Finally, this single-dose study was conducted at a specialized research center in a young healthy population, which may not completely generalize to a broader consumer population.\n\nThe improved methods used in our study have enabled us to establish that a single, non-prescription dose of NAPSO can provide significantly greater and longer-lasting pain relief over a 24-h period compared with IBU. The reduced need for rescue medication with NAPSO should enhance patient compliance and convenience. Safety assessments did not raise any unexpected findings, and there was no significant difference in the safety profile of non-prescription doses of NAPSO and IBU compared with placebo, confirming previous findings that over-the-counter NAPSO has a good safety profil[e22](#page-10-0),[23](#page-10-0).\n\n### Conclusions\n\nEvaluation of a single dose of two commonly-used NSAIDs over a 24-h period indicates that analgesic effectiveness lasts significantly longer with NAPSO 440 mg than with IBU 400 mg in post-surgical dental pain. Although the NSAIDs had similar analgesic efficacy over the first 6–7 h and a similar peak level of analgesia, pain relief was sustained for a longer time with NAPSO than IBU, and substantially fewer NAPSO patients required any rescue medication. Both NAPSO 440 mg and IBU 400 mg were safe and well tolerated.\n\n### Note\n\n1. SAS Institute Inc., Cary, NC, USA.\n\n### Transparency section\n\n### Declaration of funding\n\nThis study was sponsored by Bayer HealthCare LLC, Consumer Health, Whippany, NJ.\n\n### Author contributions\n\nAll authors were involved in the research plan, analysis and interpretation of the data, drafting of the manuscript and revising it critically for intellectual content, and final approval of the version to be published. All authors are accountable for all aspects of the work.\n\n### Declarations of financial/other relationships\n\nAlberto Paredes-Diaz, Emanuel Troullos, Robert Centofanti, and Bob An are employees of Bayer Consumer Health, Stephen A. Cooper and Paul Desjardins are consultants to Bayer Healthcare, and Patrick Brain is employed by JBR Clinical Research. Peer reviewers on this manuscript have received an honorarium from CMRO for their review work but have no other relevant financial relationships to disclose.\n\n# <span id=\"page-10-0\"></span>Acknowledgements\n\nDeborah Nock (Medical WriteAway, Norwich, UK), a medical writer, drafted and edited the manuscript, with full review and approval by all authors at every stage.\n\n## References\n\n- [\\[1\\] v](#page-1-0)an Tulder M, Becker A, Bekkering T, et al. European guidelines for the management of acute nonspecific low back pain in primary care. Eur Spine J. 2006;15:S169–S191.\n- [\\[2\\] A](#page-9-0)merican Dental Association. Statement on the use of opioids in the treatment of dental pain; 2016 [cited 2019 Jan 28]. Available from: [https://www.ada.org/en/advocacy/current-policies/sub](https://www.ada.org/en/advocacy/current-policies/substance-use-disorders)[stance-use-disorders](https://www.ada.org/en/advocacy/current-policies/substance-use-disorders)\n- [3] Alaska Dental Association. For safe prescribing of opioids and nonopiate alternatives; 2018 [cited 2018 Nov 21]. Available from: [https://www.akdental.org/docs/librariesprovider9/opioid/11-18-ads](https://www.akdental.org/docs/librariesprovider9/opioid/11-18-ads-opioid-amp-nonopioid-alternative-prescribing-guideline.pdf)[opioid-amp-nonopioid-alternative-prescribing-guideline.pdf](https://www.akdental.org/docs/librariesprovider9/opioid/11-18-ads-opioid-amp-nonopioid-alternative-prescribing-guideline.pdf)\n- [4] Kloppenburg M, Kroon FPB, Blanco FJ, et al. 2018 update of the EULAR recommendations for the management of hand osteoarthritis. Ann Rheum Dis. 2018;78:16–24.\n- [5] Bruyere O, Cooper C, Pelletier JP, et al. A consensus statement on the European Society for Clinical and Economic Aspects of Osteoporosis and Osteoarthritis (ESCEO) algorithm for the management of knee osteoarthritis—From evidence-based medicine to the real-life setting. Semin Arthritis Rheum. 2016;45:S3–S11.\n- [\\[6\\] E](#page-1-0)vers S, Afra J, Frese A, et al. EFNS guideline on the drug treatment of migraine-revised report of an EFNS task force. Eur J Neurol. 2009;16:968–981.\n- [7] Bendtsen L, Evers S, Linde M, et al. EFNS guideline on the treatment of tension-type headache - Report of an EFNS task force. Eur J Neurol. 2010;17:1318–1325.\n- [8] Scottish Dental Clinical Effectiveness Programme. Drug prescribing for dentistry. Dental clinical guidelines; [Internet] 2016. Available from: [http://www.sdcep.org.uk/wp-content/uploads/](http://www.sdcep.org.uk/wp-content/uploads/2016/03/SDCEP-Drug-Prescribing-for-Dentistry-3rd-edition.pdf) [2016/03/SDCEP-Drug-Prescribing-for-Dentistry-3rd-edition.pdf](http://www.sdcep.org.uk/wp-content/uploads/2016/03/SDCEP-Drug-Prescribing-for-Dentistry-3rd-edition.pdf)\n- [9] G eraud G, Lant eri-Minet M, Lucas C, et al. French guidelines for the diagnosis and management of migraine in adults and children. Clin Ther. 2004;26:1305–1308.\n- [10] National Institute for Health and Care Excellence (NICE). Dysmenorrhea; 2014 [cited 2017 Nov 24]. Available from: [https://](https://cks.nice.org.uk/dysmenorrhoea) [cks.nice.org.uk/dysmenorrhoea](https://cks.nice.org.uk/dysmenorrhoea)\n\n- [\\[11\\] B](#page-1-0)rune K. Persistence of NSAIDs at effect sites and rapid disappearance from side-effect compartments contributes to tolerability. Curr Med Res Opinion. 2007;23:2985–2995.\n- [\\[12\\] K](#page-1-0)iersch T, Halladay SC, Hormel P. A single-dose, double-blind comparison of naproxen sodium, acetaminophen, and placebo in postoperative dental pain. Clin Ther. 1994;16:394–404.\n- [\\[13\\] C](#page-1-0)ooper SA, Desjardins PJ, Turk DC, et al. Research design considerations for single-dose analgesic clinical trials in acute pain: IMMPACT recommendations. Pain. 2016;157:288–301.\n- [14] Cooper SA. Models for clinical assessment of oral analgesics. Am J Med. 1983;75:24–29.\n- [15] Cooper SA, Desjardins PJ. The value of the dental impaction pain model in drug development. Methods Mol Biol. 2010;617: 175–190.\n- [\\[16\\] U](#page-1-0).S. Department of Health and Human Services Food and Drug Administration Center for Drug Evaluation and Research (CDER). Guidance for industry analgesic indications: developing drug and biological products; 2014 [cited 2018 Dec 23]. Available from: [https://www.fda.gov/downloads/drugs/guidancecomplianceregu](https://www.fda.gov/downloads/drugs/guidancecomplianceregulatoryinformation/guidances/ucm384691.pdf)[latoryinformation/guidances/ucm384691.pdf.](https://www.fda.gov/downloads/drugs/guidancecomplianceregulatoryinformation/guidances/ucm384691.pdf)\n- [\\[17\\] K](#page-1-0)iersch T, Halladay SC, Koschik M. A double-blind, randomized study of naproxen sodium, ibuprofen, and placebo in postoperative dental pain. Clin Ther. 1993;15:845–854.\n- [\\[18\\] F](#page-1-0)ricke JR, Halladay SC, Francisco CA. Efficacy and safety of naproxen sodium and ibuprofen for pain relief after oral surgery. Curr Ther Res. 1993;54:619–627.\n- [\\[19\\] S](#page-2-0)ingla NK, Desjardins PJ, Chang PD. A comparison of the clinical and experimental characteristics of four acute surgical pain models: dental extraction, bunionectomy, joint replacement, and soft tissue surgery. Pain. 2014;155:441–456.\n- [\\[20\\] J](#page-2-0)ung YH, Cho BH. Radiographic evaluation of third molar development in 6- to 24-year-olds. Imaging Sci Dent. 2014;44:185–191.\n- [\\[21\\] M](#page-9-0)oore PA, Ziegler KM, Lipman RD, et al. Benefits and harms associated with analgesic medications used in the management of acute dental pain: an overview of systematic reviews. J Am Dent Assoc. 2018;149:256.e3–265.e3.\n- [\\[22\\] B](#page-9-0)ansal V, Dex T, Proskin H, et al. A look at the safety profile of over-the-counter naproxen sodium: a meta-analysis. J Clin Pharmacol. 2001;41:127–138.\n- [\\[23\\] D](#page-9-0)erry C, Derry S, Moore R, et al. Single dose oral naproxen and naproxen sodium for acute postoperative pain in adults. Cochrane Database Syst Rev. 2009;(1)CD004234.
"""
    out = integrate_table_extraction(md)
    print(json.dumps(out, indent=2))


{
  "total_tables": 5,
  "tables": [
    {
      "id": "table_1",
      "title": null,
      "caption": "**To link to this article:** <https://doi.org/10.1080/03007995.2019.1655257>",
      "headers": [
        "View supplementary material",
        "Published online: 27 Aug 2019."
      ],
      "rows": [
        [
          "Submit your article to this journal",
          "Article views: 783"
        ]
      ],
      "structure": {
        "columns": 2,
        "rows": 1,
        "has_numeric_data": false,
        "numeric_column_indices": []
      }
    },
    {
      "id": "table_2",
      "title": null,
      "caption": "The reduction in pain intensity over 24 h (SPID0-24) was significantly greater with NAPSO (LSMean 83.30) than with",
      "headers": [
        "",
        "",
        "Table 1. Subject demographics and characteristics (per protocol population).",
        "",
        "",
        "",
        "",
        ""
      ],
      "rows": [
        [
          "--",
        

In [5]:
import pandas as pd
import json
import os
from typing import Dict, List

def excel_to_json_converter(excel_file_path: str, output_directory: str = "output_jsons"):
    """
    Convert Excel file with 4 MASTER sheets to 4 JSON files using predefined meaningful headers.
    Adds a 'filename' field to each row based on 'Ref_ID_First_Author'.
    
    Args:
        excel_file_path (str): Path to the Excel file
        output_directory (str): Directory to save JSON files
    """
    
    # Create output directory if it doesn't exist
    os.makedirs(output_directory, exist_ok=True)
    
    # Define column headers for each sheet
    headers_mapping = get_headers_mapping()
    
    # Define the sheet mappings
    sheet_mappings = {
        'Trial Characteristics MASTER': 'trail_characteristics.json',
        'Dichotomous Outcomes MASTER': 'dichotomous_outcomes.json',
        'Continuous Outcomes MASTER': 'continuous_outcomes.json',
        'Risk of Bias MASTER': 'risk_of_bias.json'
    }
    
    # Process each MASTER sheet
    for sheet_name, json_filename in sheet_mappings.items():
        try:
            print(f"Processing sheet: {sheet_name}")
            
            # Get headers for this sheet
            if sheet_name not in headers_mapping:
                print(f"Warning: No headers defined for sheet '{sheet_name}', skipping...")
                continue
                
            headers = headers_mapping[sheet_name]
            
            # Read Excel sheet data starting from the appropriate row
            df = read_excel_with_custom_headers(excel_file_path, sheet_name, headers)
            
            if df.empty:
                print(f"Warning: No data found in sheet '{sheet_name}'")
                continue
            
            # Add filename field
            df = add_filename_field(df)
            
            # Convert to JSON
            json_data = convert_to_json(df)
            
            # Save to file
            output_path = os.path.join(output_directory, json_filename)
            save_json(json_data, output_path)
            
            print(f"Saved {len(json_data)} records to {output_path}")
            
        except Exception as e:
            print(f"Error processing sheet '{sheet_name}': {e}")
            continue

def get_headers_mapping() -> Dict[str, List[str]]:
    """
    Define meaningful column headers for each sheet.
    """
    
    # Trail Characteristics headers (provided by user)
    trail_characteristics_headers = [
        "Ref_ID", "First_Author", "Year", "Trial_Name", "Design", "Funding", 
        "Outcomes_Studied", "Outcomes_Reported", "Age_Measure_Central_Tendency",
        "Age_Central_Tendency", "Age_Measure_Variability", "Age_Variability",
        "Age_Range_Years", "Gender_Female_N", "Gender_Female_Percent", "Country",
        "Comments", "Intervention_1_Name", "Intervention_1_Description", 
        "Intervention_1_N_Randomized", "Intervention_1_Age_Measure_Central_Tendency",
        "Intervention_1_Age_Central_Tendency", "Intervention_1_Age_Measure_Variability",
        "Intervention_1_Age_Variability", "Intervention_1_Age_Range",
        "Intervention_1_Female_N", "Intervention_1_Female_Percent", "Intervention_1_Comment",
        "Intervention_2_Name", "Intervention_2_Description", "Intervention_2_N_Randomized", 
        "Intervention_2_Age_Measure_Central_Tendency", "Intervention_2_Age_Central_Tendency",
        "Intervention_2_Age_Measure_Variability", "Intervention_2_Age_Variability",
        "Intervention_2_Age_Range", "Intervention_2_Female_N", "Intervention_2_Female_Percent",
        "Intervention_2_Comment", "Intervention_3_Name", "Intervention_3_Description",
        "Intervention_3_N_Randomized", "Intervention_3_Age_Measure_Central_Tendency", 
        "Intervention_3_Age_Central_Tendency", "Intervention_3_Age_Measure_Variability",
        "Intervention_3_Age_Variability", "Intervention_3_Age_Range", "Intervention_3_Female_N",
        "Intervention_3_Female_Percent", "Intervention_3_Comment", "Intervention_4_Name",
        "Intervention_4_Description", "Intervention_4_N_Randomized", 
        "Intervention_4_Age_Measure_Central_Tendency", "Intervention_4_Age_Central_Tendency", 
        "Intervention_4_Age_Measure_Variability", "Intervention_4_Age_Variability",
        "Intervention_4_Age_Range", "Intervention_4_Female_N", "Intervention_4_Female_Percent",
        "Intervention_4_Comment", "Intervention_5_Name", "Intervention_5_Description", 
        "Intervention_5_N_Randomized", "Intervention_5_Age_Measure_Central_Tendency",
        "Intervention_5_Age_Central_Tendency", "Intervention_5_Age_Measure_Variability",
        "Intervention_5_Age_Variability", "Intervention_5_Age_Range", "Intervention_5_Female_N", 
        "Intervention_5_Female_Percent", "Intervention_5_Comment", "Intervention_6_Name",
        "Intervention_6_Description", "Intervention_6_N_Randomized", 
        "Intervention_6_Age_Measure_Central_Tendency", "Intervention_6_Age_Central_Tendency",
        "Intervention_6_Age_Measure_Variability", "Intervention_6_Age_Variability",
        "Intervention_6_Age_Range", "Intervention_6_Female_N", "Intervention_6_Female_Percent"
    ]
    
    # TODO: Add headers for other sheets when provided
    dichotomous_outcomes_headers = [
        "Ref_ID", "First_Author", "Trial_Name", "Population", "Intervention_Code", 
        "Intervention_Description", "Outcome_Type", "Outcome_Other_Specify",
        "Follow_Up_Time", "N_Analyzed", "Adverse_Effect_Specify", 
        "Adverse_Effects_All_Study", "N_Events_Number", "N_Events_Percentage",
        "Comments"
        # TODO: Complete this list when user provides headers
    ]
    
    continuous_outcomes_headers = [
        "Ref_ID", "First_Author", "Trial_Name", "Population", "Intervention_Code",
        "Intervention_Description", "Outcome_Type", "Outcome_Other_Specify",
        "Pain_Relief_Scale_Name", "Pain_Relief_Scale_Type", "Pain_Relief_Scale_Range",
        "Pain_Relief_Scale_Questions", "Global_Efficacy_Scale_Name", 
        "Global_Efficacy_Scale_Type", "Global_Efficacy_Scale_Range",
        "Global_Efficacy_Scale_Questions", "TOTPAR_Definition", "TOTPAR_Scale_Name",
        "TOTPAR_Scale_Type", "TOTPAR_Scale_Range", "TOTPAR_Scale_Questions",
        "SPID_Definition", "SPID_Scale_Name", "SPID_Scale_Type", "SPID_Scale_Range",
        "SPID_Scale_Questions", "N_Analyzed", "Central_Tendency_Type", 
        "Central_Tendency_Value", "Variability_Type", "Variability_Value",
        "Other_Results", "Comments"
        # TODO: Complete this list when user provides headers
    ]
    
    risk_of_bias_headers = [
        "Ref_ID", "First_Author", "Trial_Name", "Dichotomous_Outcome",
        "Continuous_Outcome", "Outcome_Other_Specify", "Randomization_Bias_Score",
        "Randomization_Bias_Justification", "Deviation_Bias_Score", 
        "Deviation_Bias_Justification", "Missing_Data_Bias_Score",
        "Missing_Data_Bias_Justification", "Measurement_Bias_Score",
        "Measurement_Bias_Justification", "Selection_Bias_Score",
        "Selection_Bias_Justification", "Comments"
        # TODO: Complete this list when user provides headers
    ]
    
    return {
        'Trial Characteristics MASTER': trail_characteristics_headers,
        'Dichotomous Outcomes MASTER': dichotomous_outcomes_headers,
        'Continuous Outcomes MASTER': continuous_outcomes_headers,
        'Risk of Bias MASTER': risk_of_bias_headers
    }

def read_excel_with_custom_headers(excel_file_path: str, sheet_name: str, headers: List[str]) -> pd.DataFrame:
    """
    Read Excel sheet and apply custom headers, automatically detecting data start row.
    """
    
    # Read raw data without headers
    raw_data = pd.read_excel(excel_file_path, sheet_name=sheet_name, header=None, engine='openpyxl')
    
    if raw_data.empty:
        return pd.DataFrame()
    
    # Find the row where actual data starts (first numeric Ref ID)
    data_start_row = find_data_start_row(raw_data)
    
    # Extract data starting from data_start_row
    data_rows = raw_data.iloc[data_start_row:].values.tolist()
    
    # Filter out completely empty rows
    data_rows = [row for row in data_rows if any(str(cell).strip() for cell in row[:3] if cell is not None)]
    
    if not data_rows:
        return pd.DataFrame()
    
    # Create DataFrame with custom headers
    # Ensure we don't have more headers than columns
    max_cols = max(len(row) for row in data_rows) if data_rows else 0
    adjusted_headers = headers[:max_cols] if max_cols < len(headers) else headers
    
    # If we have more columns than headers, add generic names
    if max_cols > len(adjusted_headers):
        for i in range(len(adjusted_headers), max_cols):
            adjusted_headers.append(f"Column_{i+1}")
    
    # Create DataFrame
    df = pd.DataFrame(data_rows, columns=adjusted_headers[:max_cols])
    
    # Clean the dataframe
    df = clean_dataframe(df)
    
    return df

def find_data_start_row(raw_data: pd.DataFrame) -> int:
    """
    Find the row where actual data starts (first row with numeric Ref ID).
    """
    data_rows = raw_data.values.tolist()
    
    for row_idx, row in enumerate(data_rows):
        if row and len(row) > 0:
            first_cell = str(row[0]).strip()
            # Check if it's a number (Ref ID)
            try:
                float(first_cell)
                return row_idx
            except (ValueError, TypeError):
                continue
    
    # If no numeric Ref ID found, assume data starts after row 5
    return min(5, len(data_rows))

def clean_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """
    Clean the dataframe by removing empty rows and handling missing values.
    """
    if df.empty:
        return df
    
    # Drop rows where first column (Ref ID) is empty or invalid
    first_col = df.columns[0]
    
    # Remove rows where Ref ID is NaN or empty
    df = df.dropna(subset=[first_col])
    df = df[df[first_col].astype(str).str.strip() != '']
    
    # Keep only rows where first column looks like a Ref ID (numeric)
    def is_valid_ref_id(val):
        try:
            float(str(val))
            return True
        except (ValueError, TypeError):
            return False
    
    df = df[df[first_col].apply(is_valid_ref_id)]
    
    # Reset index
    df = df.reset_index(drop=True)
    
    # Replace NaN values with empty strings
    df = df.fillna('')
    
    return df

def add_filename_field(df: pd.DataFrame) -> pd.DataFrame:
    """
    Add a 'filename' field based on 'Ref_ID_First_Author' format.
    """
    if df.empty or len(df.columns) < 2:
        df['filename'] = 'unknown'
        return df
    
    # Use first two columns (Ref_ID and First_Author)
    ref_id_col = df.columns[0]
    author_col = df.columns[1]
    
    def create_filename(row):
        ref_id = str(row[ref_id_col]).strip()
        author = str(row[author_col]).strip()
        
        # Clean author name
        import re
        author_clean = re.sub(r'[^\w\s]', '', author)
        author_clean = re.sub(r'\s+', '_', author_clean.strip())
        
        return f"{int(float(ref_id))}_{author_clean}"

    
    df['filename'] = df.apply(create_filename, axis=1)
    
    return df

def convert_to_json(df: pd.DataFrame) -> List[Dict]:
    """
    Convert DataFrame to list of dictionaries for JSON export.
    """
    records = df.to_dict('records')
    
    # Clean up the records
    cleaned_records = []
    for record in records:
        cleaned_record = {}
        for key, value in record.items():
            if pd.isna(value):
                cleaned_record[key] = None
            elif value == '':
                cleaned_record[key] = ''
            else:
                cleaned_record[key] = str(value) if not isinstance(value, (int, float, bool)) else value
        cleaned_records.append(cleaned_record)
    
    return cleaned_records

def save_json(data: List[Dict], output_path: str):
    """
    Save data to JSON file with proper formatting.
    """
    try:
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=2, ensure_ascii=False, default=str)
        print(f"Successfully saved JSON file: {output_path}")
    except Exception as e:
        print(f"Error saving JSON file {output_path}: {e}")

def preview_sheet_structure(excel_file_path: str, sheet_name: str):
    """
    Preview the structure of a sheet to help determine data start row.
    """
    try:
        raw_data = pd.read_excel(excel_file_path, sheet_name=sheet_name, header=None, engine='openpyxl')
        
        print(f"\n=== Structure Preview: {sheet_name} ===")
        data_rows = raw_data.values.tolist()
        
        for i in range(min(6, len(data_rows))):
            print(f"Row {i}: {[str(cell)[:30] + '...' if len(str(cell)) > 30 else str(cell) for cell in data_rows[i][:6]]}")
        
        data_start = find_data_start_row(raw_data)
        print(f"Data starts at row: {data_start}")
        
    except Exception as e:
        print(f"Error previewing sheet: {e}")

# Example usage
if __name__ == "__main__":
    # Method 1: Use your custom paths
    excel_to_json_converter(
        "/nlp/data/karthik9/Sprint1/Dental/csvs/Acute Dental Pain Data Extraction (NMA and Pediatrics) MASTER_ASB 1 (1).xlsx", 
        "/nlp/data/karthik9/Sprint1/Dental/jsons"
    )
    
    # Method 2: Or use default paths (commented out)
    # excel_file_path = "Acute Dental Pain Data Extraction NMA and Pediatrics MASTER_ASB 1 1.xlsx"
    # excel_to_json_converter(excel_file_path)
    
    # Optional: Preview sheet structures before conversion
    # print("=== SHEET STRUCTURE PREVIEW ===")
    # for sheet in ['Trial Characteristics MASTER', 'Dichotomous Outcomes MASTER']:
    #     preview_sheet_structure(excel_file_path, sheet)
    
    print("\nConversion complete!")
    print("Output files:")
    print("- trail_characteristics.json")
    print("- dichotomous_outcomes.json") 
    print("- continuous_outcomes.json")
    print("- risk_of_bias.json")
    
    print("\n🎉 ALL SHEETS COMPLETED! 🎉")
    print("All 4 sheets now have meaningful column headers:")
    print("✅ Trail Characteristics MASTER - 82 columns")
    print("✅ Dichotomous Outcomes MASTER - 82 columns (15 main + 67 reserved)")
    print("✅ Continuous Outcomes MASTER - 33 columns") 
    print("✅ Risk of Bias MASTER - 17 columns (5 RoB domains)")
    print("\nNote: Risk of Bias headers are based on Excel file analysis.")
    print("The CSV provided appeared to contain Continuous Outcomes data instead.")

Processing sheet: Trial Characteristics MASTER


/tmp/ipykernel_18389/146059462.py:236: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna('')


Successfully saved JSON file: /nlp/data/karthik9/Sprint1/Dental/jsons/trail_characteristics.json
Saved 87 records to /nlp/data/karthik9/Sprint1/Dental/jsons/trail_characteristics.json
Processing sheet: Dichotomous Outcomes MASTER


/tmp/ipykernel_18389/146059462.py:236: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna('')


Successfully saved JSON file: /nlp/data/karthik9/Sprint1/Dental/jsons/dichotomous_outcomes.json
Saved 1691 records to /nlp/data/karthik9/Sprint1/Dental/jsons/dichotomous_outcomes.json
Processing sheet: Continuous Outcomes MASTER


/tmp/ipykernel_18389/146059462.py:236: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna('')


Successfully saved JSON file: /nlp/data/karthik9/Sprint1/Dental/jsons/continuous_outcomes.json
Saved 723 records to /nlp/data/karthik9/Sprint1/Dental/jsons/continuous_outcomes.json
Processing sheet: Risk of Bias MASTER
Successfully saved JSON file: /nlp/data/karthik9/Sprint1/Dental/jsons/risk_of_bias.json
Saved 460 records to /nlp/data/karthik9/Sprint1/Dental/jsons/risk_of_bias.json

Conversion complete!
Output files:
- trail_characteristics.json
- dichotomous_outcomes.json
- continuous_outcomes.json
- risk_of_bias.json

🎉 ALL SHEETS COMPLETED! 🎉
All 4 sheets now have meaningful column headers:
✅ Trail Characteristics MASTER - 82 columns
✅ Dichotomous Outcomes MASTER - 82 columns (15 main + 67 reserved)
✅ Continuous Outcomes MASTER - 33 columns
✅ Risk of Bias MASTER - 17 columns (5 RoB domains)

Note: Risk of Bias headers are based on Excel file analysis.
The CSV provided appeared to contain Continuous Outcomes data instead.


/tmp/ipykernel_18389/146059462.py:236: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna('')


In [ ]:
class ExtractStudyMetadata(dspy.Signature):
    """Extract basic study metadata from medical research paper markdown.
    
    This extracts core identifying information about the dental pain management study.
    """
    
    markdown_content: str = dspy.InputField(desc="Full markdown content of the medical research paper")
    
    ref_id: str = dspy.OutputField(
        desc="Reference ID number for the study (e.g., '3844', '3848'). This is the unique identifier for the paper."
    )
    first_author: str = dspy.OutputField(
        desc="Last name of the first author (e.g., 'Cooper'). Extract only the surname."
    )
    trial_name: str = dspy.OutputField(
        desc="Name of the clinical trial if specified, or 'NR' if not reported, or empty string if not mentioned at all"
    )
    population_code: str = dspy.OutputField(
        desc="Numeric code representing the study population type. Codes: 1=simple tooth extraction, 2=surgical tooth extraction (third molar/wisdom teeth), 3=surgical tooth extraction (other teeth), 4=pulpitis or its complications. Can be multiple codes separated by commas (e.g., '2, 3')"
    )
    filename: str = dspy.OutputField(
        desc="Filename for the study in format 'RefID_FirstAuthor' (e.g., '3844_Cooper', '3848_Cooper')"
    )


class ExtractInterventions(dspy.Signature):
    """Extract intervention details from medical research paper markdown.
    
    This extracts information about pain management interventions used in dental studies.
    Focus on medication types, dosages, and participant counts.
    """
    
    markdown_content: str = dspy.InputField(desc="Full markdown content of the medical research paper")
    
    interventions_json: str = dspy.OutputField(
        desc="""JSON string containing list of interventions. Each intervention object must have:
        - intervention_code (integer): Numeric code 1-11 where:
          1=Ibuprofen 200-400mg + Acetaminophen 500-1000mg
          2=Oxycodone 5mg or Codeine 60mg  
          3=Acetaminophen 650mg + Oxycodone 10mg
          4=Ibuprofen 200mg + Hydrocodone 5mg
          5=Hydrocodone 5mg + Acetaminophen 300-325mg
          6=Ibuprofen 400mg (fast acting or acid)
          7=Tramadol 37.5mg + Acetaminophen 325mg
          8=Acetaminophen 500-1000mg
          9=Acetaminophen 600-650mg + Codeine 60mg
          10=Naproxen 400-440mg
          11=Placebo/no treatment
        - intervention_description (string): Full description with medication name and exact dose (e.g., "Ibuprofen 400mg", "Naproxen sodium 440mg")
        - n_analyzed (integer): Number of participants analyzed for this intervention group
        
        Example: [{"intervention_code": 6, "intervention_description": "Ibuprofen 400mg", "n_analyzed": 40}]"""
    )





class ExtractAllOutcomes(dspy.Signature):
    """Extract ALL outcomes from medical research paper for systematic review.
    
    This implements COMPLETE DATA CAPTURE methodology - extract every data point
    including rescue analgesia, adverse events, and other outcomes at all time points.
    Focus on dichotomous outcomes from ALL data sources: main text, figures, tables,
    and supplementary materials. Include zero-event outcomes (0/N patients).
    """
    
    markdown_content: str = dspy.InputField(desc="Full markdown content of the medical research paper including supplementary materials")
    intervention_description: str = dspy.InputField(desc="Specific intervention to extract outcomes for (e.g., 'Ibuprofen 400mg', 'Placebo')")
    
    all_outcomes_json: str = dspy.OutputField(
        desc="""JSON string containing list of ALL outcomes for the specified intervention. Each outcome object must have:
        
        MANDATORY FIELDS FOR ALL OUTCOMES:
        - outcome_type (integer): Outcome type code where:
          1=Rescue analgesia at 6 hours
          2=Rescue analgesia at 4 hours  
          4=Rescue analgesia for pulpitis population
          5=Adverse effects (nausea, vomiting, drowsiness, dizziness, headache, etc.)
          6=Other outcomes (pain relief, time to onset, etc.)
        - follow_up_time (string): Exact time point when outcome was measured (e.g., "6 hours", "24hrs", "4 hours", "7 days")
        - n_analyzed (integer): Number of participants analyzed for this specific outcome
        - n_events_number (integer): Number of patients who experienced this outcome
        - n_events_percentage (float): Percentage of patients who experienced this outcome (e.g., 17.5, 0.6, 2.4, 0.0)
        
        CONDITIONAL FIELDS:
        - adverse_effect_specify (string): Specific adverse effect name if outcome_type=5 (e.g., "Drowsiness (sleepy, tired)", "Paraesthesia oral", "Vomiting"). Use "NA" if outcome_type≠5
        - other_outcome_specify (string): Detailed description if outcome_type=6 (e.g., "Time to meaningful pain relief", "Pain intensity difference"). Use "NA" if outcome_type≠6
        - adverse_effects_all_study (string): List of all adverse effects if not reported per study arm, or "NA" if reported per arm
        
        DOCUMENTATION FIELDS:
        - extraction_notes (string): Technical documentation including data source ("From Table 2", "From Figure 5", "From Supplementary Table 3"), extraction method ("Direct from table", "Interpreted from Kaplan-Meier curve"), and population used ("Per-protocol population", "Safety population", "ITT population")
        - comments (string): Study-specific information including single vs multiple dose design, surgical techniques mentioned, methodological features, dropout rates, calculation details
        
        EXTRACTION REQUIREMENTS:
        - Extract EVERY outcome reported, including zero-event outcomes (0/N)
        - Create separate entries for each time point assessment
        - Include outcomes from ALL data sources (main text, figures, supplements)
        - Use appropriate analysis populations (efficacy vs safety)
        - Document any calculations or interpretations performed
        
        Example: [
          {"outcome_type": 1, "follow_up_time": "6 hours", "n_analyzed": 40, "n_events_number": 15, "n_events_percentage": 37.5, "adverse_effect_specify": "NA", "other_outcome_specify": "NA", "adverse_effects_all_study": "NA", "extraction_notes": "From Table 3, per-protocol population", "comments": "single dose study with overnight monitoring"},
          {"outcome_type": 5, "follow_up_time": "24 hours", "n_analyzed": 40, "n_events_number": 7, "n_events_percentage": 17.5, "adverse_effect_specify": "Drowsiness (sleepy, tired)", "other_outcome_specify": "NA", "adverse_effects_all_study": "NA", "extraction_notes": "From safety table, safety population", "comments": "mild to moderate severity"}
        ]"""
    )


class StructureComprehensiveOutcome(dspy.Signature):
    """Structure extracted data into the final comprehensive dichotomous outcome format.
    
    This combines study metadata, intervention details, and any outcome data (rescue analgesia,
    adverse events, or other outcomes) into the standardized format used for systematic review
    and meta-analysis. Each record represents one outcome for one intervention in one study.
    """
    
    study_metadata_json: str = dspy.InputField(desc="Study metadata as JSON string with ref_id, first_author, trial_name, population_code, filename")
    intervention_json: str = dspy.InputField(desc="Single intervention details as JSON string with intervention_code, intervention_description, n_analyzed")
    outcome_json: str = dspy.InputField(desc="Single outcome details as JSON string with all outcome fields including outcome_type, follow_up_time, n_events_number, etc.")
    
    structured_record_json: str = dspy.OutputField(
        desc="""Complete structured record as JSON string with exactly these fields:
        - Ref_ID (float): Reference ID from study metadata (e.g., 3844.0)
        - First_Author (string): First author last name (e.g., "Cooper")
        - Trial_Name (string): Trial name or empty string
        - Population (integer): Population code (1-4)
        - Intervention_Code (integer): Intervention code (1-11)
        - Intervention_Description (string): Full intervention description with dose
        - Outcome_Type (integer): Outcome type (1=rescue analgesia 6h, 2=rescue analgesia 4h, 4=rescue analgesia pulpitis, 5=adverse effects, 6=other)
        - Outcome_Other_Specify (string): Detailed outcome description for type 6, or empty string for other types
        - Follow_Up_Time (string): Time point (e.g., "24hrs", "6 hours")
        - N_Analyzed (integer): Number of participants analyzed
        - Adverse_Effect_Specify (string): Specific adverse effect name for type 5, or empty string for other types
        - Adverse_Effects_All_Study (string): All study adverse effects if not reported per arm, or empty string
        - N_Events_Number (integer): Number of patients with this outcome
        - N_Events_Percentage (float): Percentage of patients with this outcome
        - Comments (string): Study-specific methodology, design notes, and extraction details
        - filename (string): Study filename in format "RefID_Author"
        
        FIELD MAPPING RULES:
        - For outcome_type 1,2,4 (rescue analgesia): Adverse_Effect_Specify="" and Outcome_Other_Specify=""
        - For outcome_type 5 (adverse effects): Outcome_Other_Specify="" and Adverse_Effect_Specify=specific adverse event name
        - For outcome_type 6 (other outcomes): Adverse_Effect_Specify="" and Outcome_Other_Specify=detailed outcome description
        - Always include extraction methodology and data source information in Comments
        - Ensure mathematical validation: (N_Events_Number/N_Analyzed)*100 = N_Events_Percentage
        - Use appropriate analysis populations (efficacy vs safety) based on outcome type
        
        Example: {"Ref_ID": 3844, "First_Author": "Cooper", "Trial_Name": "", "Population": 2, "Intervention_Code": 10, "Intervention_Description": "Naproxen sodium 440mg", "Outcome_Type": 5, "Outcome_Other_Specify": "", "Follow_Up_Time": "24hrs", "N_Analyzed": 166, "Adverse_Effect_Specify": "Paraesthesia oral", "Adverse_Effects_All_Study": "", "N_Events_Number": 1, "N_Events_Percentage": 0.6, "Comments": "extracted from supplementary table 3, safety population, single dose study", "filename": "3844_Cooper"}"""
    )

In [ ]:
 class ExtractAdverseEvents(dspy.Signature):
#     """Extract adverse events and their frequencies from medical research paper markdown.
    
#     This extracts specific adverse effects reported for a given intervention, including
#     frequencies, percentages, and follow-up timing. Focus on side effects like nausea,
#     vomiting, drowsiness, dizziness, headache, etc.
#     """
    
#     markdown_content: str = dspy.InputField(desc="Full markdown content of the medical research paper")
#     intervention_description: str = dspy.InputField(desc="Specific intervention to extract adverse events for (e.g., 'Ibuprofen 400mg')")
    
#     adverse_events_json: str = dspy.OutputField(
#         desc="""JSON string containing list of adverse events for the specified intervention. Each adverse event object must have:
#         - adverse_effect_specify (string): Specific adverse effect name (e.g., "Drowsiness (sleepy, tired)", "Paraesthesia oral", "Vomiting", "Dizziness", "Headache")
#         - n_events_number (integer): Number of patients who experienced this adverse event
#         - n_events_percentage (float): Percentage of patients who experienced this adverse event (e.g., 17.5, 0.6, 2.4)
#         - follow_up_time (string): Time point when adverse events were measured (e.g., "6 hours", "24hrs", "4 hours")
#         - outcome_other_specify (string): Additional outcome specification if needed, or "NA" if not applicable
#         - adverse_effects_all_study (string): List of all adverse effects if not reported per study arm, or "NA" if reported per arm
#         - comments (string): Any additional comments about the adverse events extraction or study details
    
#         Example: [{"adverse_effect_specify": "Drowsiness (sleepy, tired)", "n_events_number": 7, "n_events_percentage": 17.5, "follow_up_time": "6 hours", "outcome_other_specify": "NA", "adverse_effects_all_study": "NA", "comments": "single dose study"}]"""
#     )


# class StructureDichotomousOutcome(dspy.Signature):
#     """Structure extracted data into the final dichotomous outcome format.
    
#     This combines study metadata, intervention details, and adverse event data into
#     the standardized format used for medical data analysis. Each record represents
#     one adverse event for one intervention in one study.
#     """
    
#     study_metadata_json: str = dspy.InputField(desc="Study metadata as JSON string with ref_id, first_author, trial_name, population_code, filename")
#     intervention_json: str = dspy.InputField(desc="Single intervention details as JSON string with intervention_code, intervention_description, n_analyzed")
#     adverse_event_json: str = dspy.InputField(desc="Single adverse event details as JSON string with all adverse event fields")
    
#     structured_record_json: str = dspy.OutputField(
#         desc="""Complete structured record as JSON string with exactly these fields:
#         - Ref_ID (float): Reference ID from study metadata (e.g., 3844.0)
#         - First_Author (string): First author last name (e.g., "Cooper")
#         - Trial_Name (string): Trial name or empty string
#         - Population (integer): Population code (1-4)
#         - Intervention_Code (integer): Intervention code (1-11)
#         - Intervention_Description (string): Full intervention description with dose
#         - Outcome_Type (integer): Always 5 for adverse effects
#         - Outcome_Other_Specify (string): Additional outcome info or empty string
#         - Follow_Up_Time (string): Time point (e.g., "24hrs", "6 hours")
#         - N_Analyzed (integer): Number of participants analyzed
#         - Adverse_Effect_Specify (string): Specific adverse effect name
#         - Adverse_Effects_All_Study (string): All study adverse effects or empty string
#         - N_Events_Number (integer): Number of patients with this adverse event
#         - N_Events_Percentage (float): Percentage of patients with this adverse event
#         - Comments (string): Additional comments
#         - filename (string): Study filename in format "RefID_Author"
        
#         Example: {"Ref_ID": 3844, "First_Author": "Cooper", "Trial_Name": "", "Population": 2, "Intervention_Code": 10, "Intervention_Description": "Naproxen sodium 440mg", "Outcome_Type": 5, "Outcome_Other_Specify": "", "Follow_Up_Time": "24hrs", "N_Analyzed": 166, "Adverse_Effect_Specify": "Paraesthesia oral", "Adverse_Effects_All_Study": "", "N_Events_Number": 1, "N_Events_Percentage": 0.6, "Comments": "extracted from supplementary table 3", "filename": "3844_Cooper"}"""
#     )

In [6]:
import json
import re

s = '''```json
{
    "type_of_scale_used": "ordinal, categorical",
    "name_of_scale_used": "5-point categorical pain relief rating (PRR) scale",
    "range_of_scale_used": "0-4",
    "copy_paste_scale": "using a 5-point categorical pain relief rating (PRR) scale (0=none, 1=a little, 2=some, 3=a lot, and 4=complete)",
    "all_scale_responses": "complete (4), a lot (3), some (2), a little (1), none (0)",
    "extraction_notes": "Extracted from the Methods section, subsection 2.2 Study Assessments and Endpoints.",
    "additional_details": "Pain relief self-assessments took place at 0.25, 0.5, 1, 1.5, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, and 12 h post-treatment. PRR stands for Pain Relief Rating."
}
```'''



def safe_json_parse(json_string, fallback=None):
    """
    Robust JSON parser wrapper with multiple recovery strategies.
    
    Args:
        json_string: The JSON string to parse
        fallback: What to return if all parsing fails (default: empty dict)
    
    Returns:
        Parsed JSON or fallback value
    """
    if fallback is None:
        fallback = {}
    
    if not json_string or not isinstance(json_string, str):
        return fallback
    
    # Strategy 1: Direct parsing
    try:
        return json.loads(json_string)
    except json.JSONDecodeError:
        pass
    
    # Strategy 2: Clean common issues and handle nested single quotes
    try:
        cleaned = json_string.strip()
        
        # Fix common escape issues
        cleaned = cleaned.replace('\n', '\\n').replace('\r', '\\r').replace('\t', '\\t')
        
        # Fix trailing commas
        cleaned = re.sub(r',(\s*[}\]])', r'\1', cleaned)
        
        # Better handling of nested single quotes - convert entire structure
        if cleaned.startswith("'") or "': '" in cleaned or "': {'" in cleaned:
            # More robust single quote to double quote conversion
            cleaned = cleaned.replace("'", '"')
            # Fix any double-double quotes that might have been created
            cleaned = cleaned.replace('""', '"')
        else:
            # Selective single quote replacement for safer conversion
            cleaned = re.sub(r"'([^']*)':", r'"\1":', cleaned)
            cleaned = re.sub(r":\s*'([^']*)'", r': "\1"', cleaned)
        
        result = json.loads(cleaned)
        
        # Handle escaped JSON inside strings (recursive check)
        if isinstance(result, str) and result.strip().startswith(("{", "[")):
            return safe_json_parse(result, fallback)
            
        return result
    except (json.JSONDecodeError, AttributeError):
        pass
    
    # Strategy 3: Try to extract key-value pairs manually
    try:
        # Extract numbers and strings with regex
        data = {}
        
        # Find numeric values
        for match in re.finditer(r'"([^"]+)":\s*(\d+(?:\.\d+)?)', json_string):
            key, value = match.groups()
            data[key] = float(value) if '.' in value else int(value)
        
        # Find string values
        for match in re.finditer(r'"([^"]+)":\s*"([^"]*)"', json_string):
            key, value = match.groups()
            data[key] = value
        
        # Find boolean values
        for match in re.finditer(r'"([^"]+)":\s*(true|false)', json_string):
            key, value = match.groups()
            data[key] = value == 'true'
        
        if data:
            return data
    except:
        pass
    
    # Strategy 4: Try to parse as single value if it looks like one
    try:
        # Check if it's just a number, string, or boolean wrapped weirdly
        stripped = json_string.strip()
        if stripped.replace('.', '').replace('-', '').isdigit():
            return float(stripped) if '.' in stripped else int(stripped)
        elif stripped.lower() in ['true', 'false']:
            return stripped.lower() == 'true'
        elif stripped.startswith('"') and stripped.endswith('"'):
            return stripped[1:-1]
    except:
        pass
    
    # All strategies failed
    return fallback

# Usage examples:
print(safe_json_parse(s))  # Normal case
# safe_json_parse('{"key": value,}')   # Handles trailing comma
# safe_json_parse("{'key': 'value'}")  # Handles single quotes
# safe_json_parse('malformed json')    # Returns fallback (empty dict)
# safe_json_parse('malformed', {"error": True})  # Custom fallback

{'type_of_scale_used': 'ordinal, categorical', 'name_of_scale_used': '5-point categorical pain relief rating (PRR) scale', 'range_of_scale_used': '0-4', 'copy_paste_scale': 'using a 5-point categorical pain relief rating (PRR) scale (0=none, 1=a little, 2=some, 3=a lot, and 4=complete)', 'all_scale_responses': 'complete (4), a lot (3), some (2), a little (1), none (0)', 'extraction_notes': 'Extracted from the Methods section, subsection 2.2 Study Assessments and Endpoints.', 'additional_details': 'Pain relief self-assessments took place at 0.25, 0.5, 1, 1.5, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, and 12 h post-treatment. PRR stands for Pain Relief Rating.'}


In [21]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Load different models
print("Loading models...")

# Original model
general_model = SentenceTransformer('all-MiniLM-L6-v2')

# Scientific models (these might take longer to download first time)
try:
    # BioBERT-based model
    biobert_model = SentenceTransformer('pritamdeka/S-BioBert-snli-multinli-stsb')
    print("✓ BioBERT model loaded")
except:
    print("✗ BioBERT model failed to load")
    biobert_model = None

try:
    # SciBERT-based model  
    scibert_model = SentenceTransformer('pritamdeka/S-Scibert-snli-multinli-stsb')
    print("✓ SciBERT model loaded")
except:
    print("✗ SciBERT model failed to load")
    scibert_model = None

# Alternative scientific models to try
try:
    # Another biomedical model
    bio_model = SentenceTransformer('dmis-lab/biobert-base-cased-v1.1')
    print("✓ Alternative bio model loaded")
except:
    print("✗ Alternative bio model failed")
    bio_model = None

def compare_models(text1, text2, models_dict):
    """Compare similarity scores across different models"""
    print(f"\nComparing: '{text1}' vs '{text2}'")
    print("-" * 50)
    
    for model_name, model in models_dict.items():
        if model is not None:
            embeddings = model.encode([text1, text2])
            similarity = cosine_similarity([embeddings[0]], [embeddings[1]])[0][0]
            print(f"{model_name:20}: {similarity:.4f}")
        else:
            print(f"{model_name:20}: Model not available")

# Test pairs
test_pairs = [
    ("RCT", "randomized controlled trial"),
    ("100mg", "100 milligrams"), 
    ("p<0.05", "statistically significant"),
    ("hypertension", "high blood pressure"),
    ("myocardial infarction", "heart attack"),
    ("COVID-19", "coronavirus disease"),
    ("24hrs", "1 day"),
    ("3hrs", "upto 3 hours"),
    ("Acetaminophen", "paracetamol"),
    ("40", "40.319856780"),
    
]

# Create models dictionary
models = {
    "General (MiniLM)": general_model,
    "BioBERT": biobert_model,
    "SciBERT": scibert_model,
    "Bio Alternative": bio_model
}

# Run comparisons
print("\n" + "="*60)
print("MODEL COMPARISON RESULTS")
print("="*60)

for text1, text2 in test_pairs:
    compare_models(text1, text2, models)

print("\n" + "="*60)
print("INSTALLATION NOTES:")
print("="*60)
print("If models failed to load, try these alternatives:")
print()
print("# For BioBERT:")
print("pip install transformers")
print("# Then use: 'dmis-lab/biobert-base-cased-v1.1'")
print()
print("# For SciBERT:")  
print("# Use: 'allenai/scibert_scivocab_uncased'")
print()
print("# Other good scientific models:")
print("# 'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext'")
print("# 'emilyalsentzer/Bio_ClinicalBERT'")

Loading models...
✓ BioBERT model loaded


No sentence-transformers model found with name dmis-lab/biobert-base-cased-v1.1. Creating a new one with mean pooling.


✓ SciBERT model loaded
✓ Alternative bio model loaded

MODEL COMPARISON RESULTS

Comparing: 'RCT' vs 'randomized controlled trial'
--------------------------------------------------
General (MiniLM)    : 0.2063
BioBERT             : 0.7684
SciBERT             : 0.7753
Bio Alternative     : 0.6639

Comparing: '100mg' vs '100 milligrams'
--------------------------------------------------
General (MiniLM)    : 0.7371
BioBERT             : 0.8117
SciBERT             : 0.8110
Bio Alternative     : 0.8726

Comparing: 'p<0.05' vs 'statistically significant'
--------------------------------------------------
General (MiniLM)    : 0.4270
BioBERT             : 0.5985
SciBERT             : 0.5288
Bio Alternative     : 0.8657

Comparing: 'hypertension' vs 'high blood pressure'
--------------------------------------------------
General (MiniLM)    : 0.7266
BioBERT             : 0.7945
SciBERT             : 0.7020
Bio Alternative     : 0.9311

Comparing: 'myocardial infarction' vs 'heart attack'
---